In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import tensorflow_datasets as tfds

%load_ext tensorboard

from os import path
import pathlib
import tempfile

In [3]:
# normalizing the images to [0, 1]
def normalize(image, label):
    return tf.cast(image, tf.float32) / 255., label

def random_crop(image):
    cropped_image = tf.image.random_crop(
        image, size=[256, 256, 3])

    return cropped_image

def random_jitter(image):
    # resizing to 286 x 286 x 3
    image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # randomly cropping to 256 x 256 x 3
    image = random_crop(image)

    # random mirroring
    image = tf.image.random_flip_left_right(image)

    return image

def preprocess_tf_flowers_train(image, label):
    image = random_jitter(image)
    return image, label

# -------------------------------

def preprocess_tf_flowers(image, label):
    image = tf.image.resize(image, [256, 256],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return image, label

def load_flowers_dataset():  
    (ds_train, ds_validation, ds_test), ds_info = tfds.load(name="oxford_flowers102", 
                                             with_info=True,
                                             split=['train', 'validation', 'test'],  
                                             as_supervised=True)

    ds_train = ds_train.map(normalize)    
    ds_train = ds_train.map(preprocess_tf_flowers)
    ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
    
    ds_validation = ds_validation.map(normalize)
    ds_validation = ds_validation.map(preprocess_tf_flowers)
    
    ds_test = ds_test.map(normalize)
    ds_test = ds_test.map(preprocess_tf_flowers)
    
    return ds_train, ds_validation, ds_test

def load_tf_flowers_dataset():  
    (ds_train, ds_validation, ds_test), ds_info = tfds.load(name="tf_flowers", 
                                             with_info=True,
                                             split=['train[:70%]', 'train[70%:85%]', 'train[85%:]'],  #70/15/15 split
                                             as_supervised=True)

    ds_train = ds_train.map(normalize)    
    ds_train = ds_train.map(preprocess_tf_flowers)
    ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
    
    ds_validation = ds_validation.map(normalize)
    ds_validation = ds_validation.map(preprocess_tf_flowers)
    
    ds_test = ds_test.map(normalize)
    ds_test = ds_test.map(preprocess_tf_flowers)
    
    return ds_train, ds_validation, ds_test

def load_beans_datasets():
    (ds_train, ds_validation, ds_test), ds_info = tfds.load(
        'beans',
        split=['train', 'validation', 'test'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True,
    )
    
    ds_train = ds_train.map(normalize)
    ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
    
    ds_validation = ds_validation.map(normalize)
    
    ds_test = ds_test.map(normalize)
    
    return ds_train, ds_validation, ds_test

# Optimization after training

## Quantization after training

1. Dynamic range quantization
2. Full integer quantization
3. Float16 quantization

### 1. Dynamic range quantization

Only the weights are converted from float to 8 bit int. 

### 2. Full integer quantization

Weights and activation outputs are quantizated. Good for microcontrolers and TPUs

In [4]:
def model_quantization(model_path, ds):
    # check if model was already in optimization folder
    if (len(model_path.split('/')[0].split('_')) == 3):
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '/')
    else:
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '_optimized/')
    model_name = model_path.split('/')[-1].split('.')[0]
    
    model = tf.keras.models.load_model(model_path)
    
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    
    # save converted tflite model
    tf_model_path = optimized_dir/(model_name + '.tflite')
    size = tf_model_path.write_bytes(tflite_model)
    print('Converted TFLite model ('+ str(size) +' Bytes) saved to: ' + str(tf_model_path))
    
    # 1. optimize model using dynamic range quantization
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model_quant = converter.convert()
    
    tf_quant_model_path = optimized_dir/(model_name + '_dynamic_rage_quantization.tflite')
    size = tf_quant_model_path.write_bytes(tflite_model_quant)
    print('Dynamic range quantizatized TFLite model ('+ str(size) +' Bytes) saved to: ' + str(tf_quant_model_path))
    
    # 2. Full integer quantization
    def representative_data_gen():
        for input_value, _ in ds.batch(1).take(100):
            # Model has only one input so each data point has one element.
            yield [input_value]

    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_data_gen

    tflite_model_quant = converter.convert()
    tf_quant_model_path = optimized_dir/(model_name + '_full_integer_quantization.tflite')
    size = tf_quant_model_path.write_bytes(tflite_model_quant)
    print('Full integer quantizatized TFLite model ('+ str(size) +' Bytes) saved to: ' + str(tf_quant_model_path))
    
    # 2.1 Full integer quantization with input and output in integer too
    try:
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_data_gen
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.uint8
        converter.inference_output_type = tf.uint8

        tflite_model_quant = converter.convert()
        tf_quant_model_path = optimized_dir/(model_name + '_full_integer_quantization_integer_io.tflite')
        size = tf_quant_model_path.write_bytes(tflite_model_quant)
        print('Full integer quantizatized with integer io TFLite model ('+ str(size) +' Bytes) saved to: ' + str(tf_quant_model_path))
    except:
        print('ERROR: Failed Full integer quantizatized with integer io TFLite model')
        
    # 3. float16 quantization
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]
    
    tflite_model_quant = converter.convert()
    tf_quant_model_path = optimized_dir/(model_name + '_float16_quantization.tflite')
    size = tf_quant_model_path.write_bytes(tflite_model_quant)
    print('float16 quantizatized TFLite model ('+ str(size) +' Bytes) saved to: ' + str(tf_quant_model_path))


## Pruning and fine-tuning

1. Prune model to different sparsity  ( tf uses magnitude-based pruning )
    1. ConstantSparsity - sparsity is kept constant during training.
    2. PolynomialDecay - the degree of sparsity is changed during training.
2. Fine-tune model

https://www.machinecurve.com/index.php/2020/09/29/tensorflow-pruning-schedules-constantsparsity-and-polynomialdecay/

In [5]:
BATCH_SIZE = 32
PRUNING_EPOCHS = 10
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

In [6]:
# Some layers cannot be pruned

def prune_prunable_layers(model, pruning_params):
    """returns model for pruning with avoided non prunable layers"""
    
    # Rescaling layer cannot be pruned
    def apply_pruning_to_prunable(layer):
        if isinstance(layer, tf.keras.layers.experimental.preprocessing.Rescaling) or isinstance(layer, tf.keras.layers.experimental.preprocessing.Normalization):
            return layer
        return prune_low_magnitude(layer, **pruning_params)
    model_for_pruning = tf.keras.models.clone_model(
                                model,
                                clone_function=apply_pruning_to_prunable,
                            )
    return model_for_pruning

In [7]:
def prune_model(model_path, batch_size, pruning_epochs, ds_train, ds_validation, sparsity):
     # check if model was already in optimization folder
    if (len(model_path.split('/')[0].split('_')) == 3):
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '/')
    else:
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '_optimized/')
    model_name = model_path.split('/')[-1].split('.')[0]
    
    model = tf.keras.models.load_model(model_path)
    
    sparsities = [0.5, 0.75, 0.9]
    
    ds_train = ds_train.batch(batch_size)
    ds_validation = ds_validation.batch(batch_size)
    
    ds_train.cache()
    ds_validation.cache()
    
    # get number of images
    num_images = 0
    for i in ds_train.as_numpy_iterator():
        num_images+=1

    end_step = np.ceil(num_images / batch_size).astype(np.int32) * pruning_epochs

    # Define pruning configuration
    pruning_params_constant = {
        'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=sparsity,
                                                                    begin_step=0,
                                                                    end_step=end_step)
    }

    pruning_params_polynomial = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0,
                                                                final_sparsity=sparsity,
                                                                begin_step=0,
                                                                end_step=end_step)
    }

    model_cs = tf.keras.models.clone_model(model)
    model_pd = tf.keras.models.clone_model(model)
    
    # Rescaling layer cannot be wrapped in prune low magnitude
    try:
        model_for_pruning_constant = prune_low_magnitude(model_cs, **pruning_params_constant)
        model_for_pruning_polynomial = prune_low_magnitude(model_pd, **pruning_params_polynomial)
    except:
        model_for_pruning_constant = prune_prunable_layers(model_cs, pruning_params_constant) 
        model_for_pruning_polynomial = prune_prunable_layers(model_pd, pruning_params_polynomial)


    # Compile models for pruning
    model_for_pruning_constant.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])
    model_for_pruning_polynomial.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])


    constant_log_dir = pathlib.Path("./tmp/" + model_name + '_ConstantSparsity' + str(int(sparsity*100)))
    constant_log_dir.mkdir(parents=True, exist_ok=True)
    polynomial_log_dir = pathlib.Path("./tmp/" + model_name + '_PolynomialDecay' + str(int(sparsity*100)))
    polynomial_log_dir.mkdir(parents=True, exist_ok=True)

    # Model callbacks
    constant_callbacks = [
        tfmot.sparsity.keras.UpdatePruningStep(),
        tfmot.sparsity.keras.PruningSummaries(log_dir=str(constant_log_dir))
    ]
    polynomial_callbacks = [
        tfmot.sparsity.keras.UpdatePruningStep(),
        tfmot.sparsity.keras.PruningSummaries(log_dir=str(polynomial_log_dir))
    ]

    # Fitting data
    model_for_pruning_constant.fit(ds_train,
                              validation_data=ds_validation,
                              epochs=pruning_epochs,
                              callbacks=constant_callbacks)

    model_for_pruning_polynomial.fit(ds_train,
                              validation_data=ds_validation,
                              epochs=pruning_epochs,
                              callbacks=polynomial_callbacks)

    # Save pruned models
    model_constant_path = optimized_dir/(model_name + '_ConstantSparsity' + str(int(sparsity*100)) + '.h5')
    model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning_constant)
    model_for_export.save(str(model_constant_path))
    print('saved ' + str(model_constant_path))

    model_polynomial_path = optimized_dir/(model_name + '_PolynomialDecay' + str(int(sparsity*100)) + '.h5')
    model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning_polynomial)
    model_for_export.save(str(model_polynomial_path))
    print('saved ' + str(model_polynomial_path))

## Weight clustering

In [8]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

In [9]:
# Some layers cannot be weight clustered

def cluster_clustred_layers(model, cluster_params):    
    # Rescaling layer cannot be pruned
    def apply_clustering_to_clusterable(layer):
        if model.layers[0] == layer or  model.layers[-1] == layer:
            return layer
        try:
            x = cluster_weights(layer, **cluster_params)
            return x
        except:
            return layer

    model_for_clustering = tf.keras.models.clone_model(
                                model,
                                clone_function=apply_clustering_to_clusterable,
                            )
    return model_for_clustering

In [10]:
def weight_cluster_model(model_path, batch_size, epochs, ds_train, ds_validation, number_of_clusters):
    """ Weight clustering on given moodel 
    note: cannot use for cycle in this function to do different number of clusters because of compatibility issues"""
     # check if model was already in optimization folder
    if (len(model_path.split('/')[0].split('_')) == 3):
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '/')
    else:
        optimized_dir = pathlib.Path(model_path.split('/')[0] + '_optimized/')
    model_name = model_path.split('/')[-1].split('.')[0]
    
    model = tf.keras.models.load_model(model_path)
    
    ds_train = ds_train.batch(batch_size)
    ds_validation = ds_validation.batch(batch_size)
    
    ds_train.cache()
    ds_validation.cache()
    
    # get number of images
    num_images = 0
    for i in ds_train.as_numpy_iterator():
        num_images+=1

    end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

    # Define weight clustering configuration
    cluster_params_kmeans = {
                                  'number_of_clusters': number_of_clusters,
                                  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
                                }

    # Rescaling layer cannot be wrapped in prune low magnitude
    try:
        model_for_clustering_kmeans = cluster_weights(model, **cluster_params_kmeans)
    except:
        model_for_clustering_kmeans = cluster_clustred_layers(model, cluster_params_kmeans)

    # Compile models for clustering
    model_for_clustering_kmeans.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

    # Fitting data        
    model_for_clustering_kmeans.fit(ds_train,
                              validation_data=ds_validation,
                              epochs=epochs)

    # Save pruned models
    model_kmeans_path = optimized_dir/(model_name + '_KMeansPlusPlus' + str(number_of_clusters) + '.h5')
    model_for_export = tfmot.clustering.keras.strip_clustering(model_for_clustering_kmeans)
    model_for_export.save(str(model_kmeans_path))
    print('saved ' + str(model_kmeans_path))

# optimze models

In [11]:
# datasets
flowers_datasets = load_flowers_dataset()
tf_flowers_datasets = load_tf_flowers_dataset()
beans_datasets = load_beans_datasets()

### Note

Due to bad tensorflow optimization of calling fit function in a loop, calling prune_model in a loop is unusable.
Solution: don't use for loops
Issue: https://github.com/tensorflow/tensorflow/issues/34025

# Only oxford flowers

In [11]:
# todo odtud
prune_model(model_path='flowers_models/MobileNetV2_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='flowers_models/MobileNetV2_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='flowers_models/MobileNetV2_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

/home/macenpav/.conda/envs/ml/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/10
 6/32 [====>.........................] - ETA: 9s - loss: 4.8381 - accuracy: 0.0028    WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0999s vs `on_train_batch_end` time: 0.2080s). Check your callbacks.


32/32 [==============================] - 45s 600ms/step - loss: 4.8316 - accuracy: 0.0127 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 26s 517ms/step - loss: 4.2679 - accuracy: 0.0583 - val_loss: 4.6255 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 25s 508ms/step - loss: 3.8345 - accuracy: 0.0861 - val_loss: 4.6271 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 27s 536ms/step - loss: 3.4922 - accuracy: 0.1036 - val_loss: 4.6310 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 26s 522ms/step - loss: 3.2784 - accuracy: 0.1798 - val_loss: 4.6390 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 26s 531ms/step - loss: 2.9270 - accuracy: 0.2400 - val_loss: 4.6527 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 26s 520ms/step - loss: 2.7867 - accuracy: 0.2470 - val_loss: 4.6714 - val_accuracy: 0.0098
Epoch 8/10

32/32 [==============================] - 47s 620ms/step - loss: 4.9733 - accuracy: 0.0187 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 27s 542ms/step - loss: 4.1998 - accuracy: 0.0709 - val_loss: 4.6270 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 26s 518ms/step - loss: 3.8128 - accuracy: 0.0999 - val_loss: 4.6355 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 26s 516ms/step - loss: 3.4648 - accuracy: 0.1237 - val_loss: 4.6588 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 27s 529ms/step - loss: 3.3726 - accuracy: 0.1420 - val_loss: 4.6901 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 25s 509ms/step - loss: 2.9196 - accuracy: 0.2130 - val_loss: 4.7612 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 25s 520ms/step - loss: 2.7869 - accuracy: 0.2299 - val_loss: 4.8475 - val_accuracy: 0.0098
Epoch 8/10

32/32 [==============================] - 43s 587ms/step - loss: 4.7269 - accuracy: 0.0151 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 25s 497ms/step - loss: 4.3269 - accuracy: 0.0495 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 26s 533ms/step - loss: 3.8916 - accuracy: 0.0797 - val_loss: 4.6254 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 25s 513ms/step - loss: 3.6154 - accuracy: 0.1078 - val_loss: 4.6260 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 26s 532ms/step - loss: 3.2835 - accuracy: 0.2037 - val_loss: 4.6273 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 26s 534ms/step - loss: 3.0053 - accuracy: 0.2262 - val_loss: 4.6294 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 26s 535ms/step - loss: 2.9183 - accuracy: 0.2177 - val_loss: 4.6327 - val_accuracy: 0.0098
Epoch 8/10

32/32 [==============================] - 47s 625ms/step - loss: 4.9860 - accuracy: 0.0155 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 26s 523ms/step - loss: 4.3535 - accuracy: 0.0412 - val_loss: 4.6273 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 26s 538ms/step - loss: 3.8514 - accuracy: 0.0805 - val_loss: 4.6379 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 26s 519ms/step - loss: 3.5391 - accuracy: 0.1232 - val_loss: 4.6587 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 26s 510ms/step - loss: 3.2921 - accuracy: 0.1770 - val_loss: 4.6941 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 26s 517ms/step - loss: 3.1739 - accuracy: 0.1770 - val_loss: 4.7434 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 26s 543ms/step - loss: 2.9019 - accuracy: 0.2268 - val_loss: 4.8111 - val_accuracy: 0.0098
Epoch 8/10

32/32 [==============================] - 44s 638ms/step - loss: 4.6651 - accuracy: 0.0131 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 25s 524ms/step - loss: 4.4289 - accuracy: 0.0401 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 27s 535ms/step - loss: 4.2493 - accuracy: 0.0472 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 26s 537ms/step - loss: 3.9225 - accuracy: 0.1003 - val_loss: 4.6253 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 26s 522ms/step - loss: 3.7224 - accuracy: 0.1294 - val_loss: 4.6256 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 25s 515ms/step - loss: 3.4913 - accuracy: 0.1602 - val_loss: 4.6261 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 26s 530ms/step - loss: 3.2872 - accuracy: 0.1759 - val_loss: 4.6269 - val_accuracy: 0.0098
Epoch 8/10

32/32 [==============================] - 47s 616ms/step - loss: 5.0254 - accuracy: 0.0107 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 26s 520ms/step - loss: 4.3052 - accuracy: 0.0493 - val_loss: 4.6274 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 26s 535ms/step - loss: 3.9287 - accuracy: 0.0759 - val_loss: 4.6388 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 26s 510ms/step - loss: 3.6379 - accuracy: 0.0870 - val_loss: 4.6674 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 27s 538ms/step - loss: 3.3329 - accuracy: 0.1688 - val_loss: 4.7138 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 26s 521ms/step - loss: 3.1947 - accuracy: 0.1687 - val_loss: 4.7953 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 26s 529ms/step - loss: 2.9801 - accuracy: 0.2087 - val_loss: 4.8738 - val_accuracy: 0.0098
Epoch 8/10

In [12]:
prune_model(model_path='flowers_models/EfficentNetB0_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='flowers_models/EfficentNetB0_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='flowers_models/EfficentNetB0_flowers_model.h5', 
            ds_train=flowers_datasets[0], 
            ds_validation=flowers_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

Epoch 1/10
 6/32 [====>.........................] - ETA: 27s - loss: 4.8031 - accuracy: 0.0000e+00WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1683s vs `on_train_batch_end` time: 0.7076s). Check your callbacks.


32/32 [==============================] - 61s 822ms/step - loss: 4.7862 - accuracy: 0.0058 - val_loss: 4.6320 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 30s 664ms/step - loss: 4.6143 - accuracy: 0.0236 - val_loss: 4.6856 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 31s 681ms/step - loss: 4.2524 - accuracy: 0.0578 - val_loss: 4.8117 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 30s 658ms/step - loss: 4.0020 - accuracy: 0.0683 - val_loss: 5.0508 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 31s 670ms/step - loss: 3.6770 - accuracy: 0.1141 - val_loss: 5.7073 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 31s 663ms/step - loss: 3.4377 - accuracy: 0.1493 - val_loss: 5.9086 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 30s 646ms/step - loss: 3.0288 - accuracy: 0.2520 - val_loss: 5.8346 - val_accuracy: 0.0098
Epoch 8/10

32/32 [==============================] - 66s 808ms/step - loss: 4.9239 - accuracy: 0.0130 - val_loss: 4.6513 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 31s 658ms/step - loss: 4.5233 - accuracy: 0.0423 - val_loss: 5.1214 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 31s 685ms/step - loss: 4.1495 - accuracy: 0.0380 - val_loss: 6.3388 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 30s 670ms/step - loss: 3.8890 - accuracy: 0.0860 - val_loss: 7.6942 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 31s 675ms/step - loss: 3.5373 - accuracy: 0.1295 - val_loss: 6.6654 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 30s 651ms/step - loss: 3.2378 - accuracy: 0.1499 - val_loss: 6.3845 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 30s 672ms/step - loss: 2.9506 - accuracy: 0.1999 - val_loss: 8.2552 - val_accuracy: 0.0098
Epoch 8/10

32/32 [==============================] - 62s 817ms/step - loss: 4.6796 - accuracy: 0.0136 - val_loss: 4.6270 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 29s 656ms/step - loss: 4.4711 - accuracy: 0.0383 - val_loss: 4.6390 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 31s 671ms/step - loss: 4.3622 - accuracy: 0.0323 - val_loss: 4.6762 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 30s 652ms/step - loss: 4.1372 - accuracy: 0.0823 - val_loss: 4.8750 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 30s 642ms/step - loss: 3.8946 - accuracy: 0.1002 - val_loss: 5.1109 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 30s 652ms/step - loss: 3.5624 - accuracy: 0.1433 - val_loss: 5.7750 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 31s 682ms/step - loss: 3.2801 - accuracy: 0.2252 - val_loss: 5.8624 - val_accuracy: 0.0098
Epoch 8/10

32/32 [==============================] - 65s 809ms/step - loss: 4.9766 - accuracy: 0.0031 - val_loss: 4.6547 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 30s 674ms/step - loss: 4.5446 - accuracy: 0.0251 - val_loss: 4.8885 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 32s 687ms/step - loss: 4.1804 - accuracy: 0.0380 - val_loss: 5.2163 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 30s 657ms/step - loss: 3.8363 - accuracy: 0.0779 - val_loss: 5.9945 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 31s 680ms/step - loss: 3.5090 - accuracy: 0.1189 - val_loss: 6.5582 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 31s 666ms/step - loss: 3.3061 - accuracy: 0.1307 - val_loss: 6.4382 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 30s 675ms/step - loss: 3.0679 - accuracy: 0.1735 - val_loss: 7.5889 - val_accuracy: 0.0098
Epoch 8/10

32/32 [==============================] - 66s 808ms/step - loss: 4.6443 - accuracy: 0.0142 - val_loss: 4.6252 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 30s 658ms/step - loss: 4.5069 - accuracy: 0.0423 - val_loss: 4.6275 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 30s 658ms/step - loss: 4.4018 - accuracy: 0.0384 - val_loss: 4.6357 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 31s 662ms/step - loss: 4.2541 - accuracy: 0.0658 - val_loss: 4.6536 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 31s 664ms/step - loss: 4.1398 - accuracy: 0.0627 - val_loss: 4.6877 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 30s 663ms/step - loss: 3.9166 - accuracy: 0.0987 - val_loss: 4.7555 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 30s 674ms/step - loss: 3.7136 - accuracy: 0.1253 - val_loss: 4.9345 - val_accuracy: 0.0098
Epoch 8/10

32/32 [==============================] - 65s 810ms/step - loss: 4.8731 - accuracy: 0.0116 - val_loss: 4.6472 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 31s 675ms/step - loss: 4.4743 - accuracy: 0.0409 - val_loss: 4.7992 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 30s 682ms/step - loss: 4.1172 - accuracy: 0.0658 - val_loss: 5.0505 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 30s 668ms/step - loss: 3.8661 - accuracy: 0.0676 - val_loss: 6.6533 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 30s 642ms/step - loss: 3.5463 - accuracy: 0.1233 - val_loss: 7.3685 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 30s 649ms/step - loss: 3.2992 - accuracy: 0.1552 - val_loss: 6.7825 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 30s 651ms/step - loss: 3.0457 - accuracy: 0.1992 - val_loss: 8.9527 - val_accuracy: 0.0098
Epoch 8/10

In [13]:
weight_cluster_model(model_path='flowers_models/MobileNetV2_flowers_model.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models/MobileNetV2_flowers_model.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
32/32 [==============================] - 32s 533ms/step - loss: 8.2054 - accuracy: 0.1695 - val_loss: 4.1890 - val_accuracy: 0.2020
Epoch 2/10
32/32 [==============================] - 26s 509ms/step - loss: 0.6522 - accuracy: 0.8379 - val_loss: 6.6768 - val_accuracy: 0.2176
Epoch 3/10
32/32 [==============================] - 26s 503ms/step - loss: 0.1261 - accuracy: 0.9644 - val_loss: 5.0541 - val_accuracy: 0.2284
Epoch 4/10
32/32 [==============================] - 26s 511ms/step - loss: 0.1162 - accuracy: 0.9578 - val_loss: 6.5619 - val_accuracy: 0.2029
Epoch 5/10
32/32 [==============================] - 26s 519ms/step - loss: 0.0952 - accuracy: 0.9755 - val_loss: 5.8236 - val_accuracy: 0.2275
Epoch 6/10
32/32 [==============================] - 26s 519ms/step - loss: 0.0575 - accuracy: 0.9799 - val_loss: 5.6476 - val_accuracy: 0.2265
Epoch 7/10
32/32 [==============================] - 25s 503ms/step - loss: 0.0611 - accuracy: 0.9818 - val_loss: 5.2441 - val_accuracy: 0.2324

In [14]:
weight_cluster_model(model_path='flowers_models/EfficentNetB0_flowers_model.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models/EfficentNetB0_flowers_model.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
32/32 [==============================] - 40s 654ms/step - loss: 0.0246 - accuracy: 0.9905 - val_loss: 7.9552 - val_accuracy: 0.1402
Epoch 2/10
32/32 [==============================] - 29s 628ms/step - loss: 0.0419 - accuracy: 0.9881 - val_loss: 6.6063 - val_accuracy: 0.1520
Epoch 3/10
32/32 [==============================] - 29s 604ms/step - loss: 0.0167 - accuracy: 0.9975 - val_loss: 8.7211 - val_accuracy: 0.1304
Epoch 4/10
32/32 [==============================] - 29s 612ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 8.4957 - val_accuracy: 0.1431
Epoch 5/10
32/32 [==============================] - 29s 621ms/step - loss: 1.9253e-04 - accuracy: 1.0000 - val_loss: 8.5434 - val_accuracy: 0.1627
Epoch 6/10
32/32 [==============================] - 29s 623ms/step - loss: 4.1650e-05 - accuracy: 1.0000 - val_loss: 8.5682 - val_accuracy: 0.1647
Epoch 7/10
32/32 [==============================] - 29s 602ms/step - loss: 1.3026e-05 - accuracy: 1.0000 - val_loss: 8.5722 - val_accu

In [15]:
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
32/32 [==============================] - 33s 552ms/step - loss: 4.7008 - accuracy: 0.0138 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 26s 516ms/step - loss: 4.6250 - accuracy: 0.0019 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 25s 521ms/step - loss: 4.6254 - accuracy: 0.0080 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 26s 512ms/step - loss: 4.6253 - accuracy: 0.0025 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 27s 539ms/step - loss: 4.6250 - accuracy: 0.0124 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 25s 524ms/step - loss: 4.6252 - accuracy: 0.0111 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 26s 513ms/step - loss: 4.6253 - accuracy: 0.0063 - val_loss: 4.6250 - val_accuracy: 0.0098

Epoch 1/10
32/32 [==============================] - 32s 516ms/step - loss: 4.6710 - accuracy: 0.0089 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 26s 520ms/step - loss: 4.6254 - accuracy: 0.0070 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 24s 498ms/step - loss: 4.6252 - accuracy: 0.0049 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 26s 512ms/step - loss: 4.6255 - accuracy: 0.0074 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 25s 502ms/step - loss: 4.6251 - accuracy: 0.0076 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 26s 513ms/step - loss: 4.6254 - accuracy: 0.0082 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 26s 513ms/step - loss: 4.6251 - accuracy: 0.0064 - val_loss: 4.6250 - val_accuracy: 0.0098

In [16]:
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
32/32 [==============================] - 33s 561ms/step - loss: 4.9436 - accuracy: 0.0143 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 25s 504ms/step - loss: 4.6255 - accuracy: 0.0131 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 25s 499ms/step - loss: 4.6252 - accuracy: 0.0022 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 25s 519ms/step - loss: 4.6253 - accuracy: 0.0038 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 26s 517ms/step - loss: 4.6254 - accuracy: 0.0102 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 26s 516ms/step - loss: 4.6254 - accuracy: 0.0092 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 26s 515ms/step - loss: 4.6255 - accuracy: 0.0152 - val_loss: 4.6250 - val_accuracy: 0.0098

Epoch 1/10
32/32 [==============================] - 32s 520ms/step - loss: 4.9444 - accuracy: 0.0081 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 26s 517ms/step - loss: 4.6251 - accuracy: 0.0049 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 26s 510ms/step - loss: 4.6251 - accuracy: 0.0096 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 25s 486ms/step - loss: 4.6250 - accuracy: 0.0102 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 25s 502ms/step - loss: 4.6251 - accuracy: 0.0093 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 26s 505ms/step - loss: 4.6251 - accuracy: 0.0113 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 25s 507ms/step - loss: 4.6252 - accuracy: 0.0126 - val_loss: 4.6250 - val_accuracy: 0.0098

In [17]:
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
32/32 [==============================] - 31s 518ms/step - loss: 4.6497 - accuracy: 0.0067 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 26s 511ms/step - loss: 4.6249 - accuracy: 0.0053 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 26s 512ms/step - loss: 4.6254 - accuracy: 0.0087 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 26s 514ms/step - loss: 4.6254 - accuracy: 0.0088 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 26s 505ms/step - loss: 4.6251 - accuracy: 0.0116 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 26s 509ms/step - loss: 4.6255 - accuracy: 0.0085 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 26s 504ms/step - loss: 4.6254 - accuracy: 0.0122 - val_loss: 4.6250 - val_accuracy: 0.0098

Epoch 1/10
32/32 [==============================] - 31s 539ms/step - loss: 4.6295 - accuracy: 0.0150 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 26s 516ms/step - loss: 4.6252 - accuracy: 0.0107 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 26s 515ms/step - loss: 4.6253 - accuracy: 0.0073 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 25s 510ms/step - loss: 4.6253 - accuracy: 0.0141 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 26s 517ms/step - loss: 4.6256 - accuracy: 0.0116 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 25s 497ms/step - loss: 4.6256 - accuracy: 0.0051 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 26s 511ms/step - loss: 4.6252 - accuracy: 0.0081 - val_loss: 4.6250 - val_accuracy: 0.0098

In [18]:
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
32/32 [==============================] - 31s 515ms/step - loss: 4.9006 - accuracy: 0.0057 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 26s 518ms/step - loss: 4.6251 - accuracy: 0.0133 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 25s 506ms/step - loss: 4.6252 - accuracy: 0.0066 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 25s 510ms/step - loss: 4.6249 - accuracy: 0.0152 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 26s 528ms/step - loss: 4.6254 - accuracy: 0.0088 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 26s 503ms/step - loss: 4.6253 - accuracy: 0.0112 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 26s 512ms/step - loss: 4.6253 - accuracy: 0.0112 - val_loss: 4.6250 - val_accuracy: 0.0098

Epoch 1/10
32/32 [==============================] - 32s 516ms/step - loss: 4.8385 - accuracy: 0.0102 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 26s 507ms/step - loss: 4.6252 - accuracy: 0.0124 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 25s 505ms/step - loss: 4.6251 - accuracy: 0.0124 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 25s 485ms/step - loss: 4.6251 - accuracy: 0.0098 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 24s 468ms/step - loss: 4.6250 - accuracy: 0.0087 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 25s 505ms/step - loss: 4.6252 - accuracy: 0.0064 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 25s 500ms/step - loss: 4.6253 - accuracy: 0.0123 - val_loss: 4.6250 - val_accuracy: 0.0098

In [19]:
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
32/32 [==============================] - 32s 531ms/step - loss: 4.6284 - accuracy: 0.0085 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 26s 525ms/step - loss: 4.6253 - accuracy: 0.0055 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 26s 524ms/step - loss: 4.6251 - accuracy: 0.0087 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 25s 504ms/step - loss: 4.6250 - accuracy: 0.0122 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 26s 524ms/step - loss: 4.6260 - accuracy: 0.0089 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 25s 501ms/step - loss: 4.6249 - accuracy: 0.0102 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 26s 512ms/step - loss: 4.6248 - accuracy: 0.0109 - val_loss: 4.6250 - val_accuracy: 0.0098

Epoch 1/10
32/32 [==============================] - 32s 544ms/step - loss: 4.6298 - accuracy: 0.0028 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 27s 540ms/step - loss: 4.6248 - accuracy: 0.0092 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 26s 507ms/step - loss: 4.6249 - accuracy: 0.0139 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 26s 511ms/step - loss: 4.6253 - accuracy: 0.0081 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 25s 499ms/step - loss: 4.6257 - accuracy: 0.0061 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 27s 532ms/step - loss: 4.6254 - accuracy: 0.0058 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 26s 525ms/step - loss: 4.6254 - accuracy: 0.0144 - val_loss: 4.6250 - val_accuracy: 0.0098

In [20]:
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/MobileNetV2_flowers_model_PolynomialDecay90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
32/32 [==============================] - 33s 533ms/step - loss: 4.9287 - accuracy: 0.0088 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 26s 513ms/step - loss: 4.6253 - accuracy: 0.0076 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 26s 518ms/step - loss: 4.6252 - accuracy: 0.0051 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 26s 523ms/step - loss: 4.6253 - accuracy: 0.0120 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 26s 512ms/step - loss: 4.6253 - accuracy: 0.0035 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 25s 499ms/step - loss: 4.6251 - accuracy: 0.0087 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 25s 501ms/step - loss: 4.6254 - accuracy: 0.0105 - val_loss: 4.6250 - val_accuracy: 0.0098

Epoch 1/10
32/32 [==============================] - 31s 543ms/step - loss: 4.9692 - accuracy: 0.0066 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 25s 491ms/step - loss: 4.6252 - accuracy: 0.0091 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 25s 499ms/step - loss: 4.6251 - accuracy: 0.0069 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 26s 515ms/step - loss: 4.6251 - accuracy: 0.0072 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 26s 523ms/step - loss: 4.6254 - accuracy: 0.0058 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 26s 525ms/step - loss: 4.6249 - accuracy: 0.0075 - val_loss: 4.6250 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 25s 503ms/step - loss: 4.6251 - accuracy: 0.0141 - val_loss: 4.6250 - val_accuracy: 0.0098

In [21]:
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
32/32 [==============================] - 39s 665ms/step - loss: 5.4539 - accuracy: 0.0078 - val_loss: 4.6478 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 30s 622ms/step - loss: 4.6535 - accuracy: 0.0114 - val_loss: 4.6409 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 29s 629ms/step - loss: 4.6466 - accuracy: 0.0057 - val_loss: 4.6323 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 29s 624ms/step - loss: 4.6373 - accuracy: 0.0036 - val_loss: 4.6269 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 29s 618ms/step - loss: 4.6466 - accuracy: 0.0016 - val_loss: 4.6296 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 29s 616ms/step - loss: 4.6331 - accuracy: 0.0058 - val_loss: 4.6239 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 29s 616ms/step - loss: 4.6356 - accuracy: 0.0076 - val_loss: 4.6309 - val_accuracy: 0.0098

Epoch 1/10
32/32 [==============================] - 39s 647ms/step - loss: 5.6166 - accuracy: 0.0051 - val_loss: 4.6632 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 29s 622ms/step - loss: 4.6658 - accuracy: 0.0102 - val_loss: 4.6338 - val_accuracy: 0.0118
Epoch 3/10
32/32 [==============================] - 29s 610ms/step - loss: 4.6516 - accuracy: 0.0050 - val_loss: 4.6417 - val_accuracy: 0.0059
Epoch 4/10
32/32 [==============================] - 29s 613ms/step - loss: 4.6491 - accuracy: 0.0085 - val_loss: 4.6248 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 29s 604ms/step - loss: 4.6161 - accuracy: 0.0062 - val_loss: 4.5721 - val_accuracy: 0.0186
Epoch 6/10
32/32 [==============================] - 29s 613ms/step - loss: 4.5817 - accuracy: 0.0079 - val_loss: 4.5058 - val_accuracy: 0.0196
Epoch 7/10
32/32 [==============================] - 29s 621ms/step - loss: 4.4002 - accuracy: 0.0100 - val_loss: 4.3023 - val_accuracy: 0.0363

In [22]:
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay50.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
32/32 [==============================] - 39s 644ms/step - loss: 5.2913 - accuracy: 0.0140 - val_loss: 4.6304 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 30s 636ms/step - loss: 4.6441 - accuracy: 0.0097 - val_loss: 4.6185 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 29s 593ms/step - loss: 4.6264 - accuracy: 0.0074 - val_loss: 4.6077 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 28s 602ms/step - loss: 4.5952 - accuracy: 0.0150 - val_loss: 4.4358 - val_accuracy: 0.0294
Epoch 5/10
32/32 [==============================] - 28s 591ms/step - loss: 4.3384 - accuracy: 0.0328 - val_loss: 4.0969 - val_accuracy: 0.0549
Epoch 6/10
32/32 [==============================] - 29s 605ms/step - loss: 3.9295 - accuracy: 0.0825 - val_loss: 3.8187 - val_accuracy: 0.0833
Epoch 7/10
32/32 [==============================] - 29s 621ms/step - loss: 3.4361 - accuracy: 0.1578 - val_loss: 3.6881 - val_accuracy: 0.1196

Epoch 1/10
32/32 [==============================] - 41s 663ms/step - loss: 5.1063 - accuracy: 0.0092 - val_loss: 4.6204 - val_accuracy: 0.0127
Epoch 2/10
32/32 [==============================] - 29s 631ms/step - loss: 4.6298 - accuracy: 0.0118 - val_loss: 4.6565 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 30s 645ms/step - loss: 4.6147 - accuracy: 0.0101 - val_loss: 4.5962 - val_accuracy: 0.0196
Epoch 4/10
32/32 [==============================] - 30s 632ms/step - loss: 4.4951 - accuracy: 0.0179 - val_loss: 4.1416 - val_accuracy: 0.0559
Epoch 5/10
32/32 [==============================] - 29s 632ms/step - loss: 3.9391 - accuracy: 0.0631 - val_loss: 3.8054 - val_accuracy: 0.0980
Epoch 6/10
32/32 [==============================] - 29s 634ms/step - loss: 3.4421 - accuracy: 0.1404 - val_loss: 3.5402 - val_accuracy: 0.1471
Epoch 7/10
32/32 [==============================] - 29s 633ms/step - loss: 3.0584 - accuracy: 0.1984 - val_loss: 3.4515 - val_accuracy: 0.1461

In [23]:
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
32/32 [==============================] - 40s 672ms/step - loss: 5.0812 - accuracy: 0.0135 - val_loss: 4.6645 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 29s 612ms/step - loss: 4.6816 - accuracy: 0.0047 - val_loss: 4.6321 - val_accuracy: 0.0108
Epoch 3/10
32/32 [==============================] - 30s 632ms/step - loss: 4.6436 - accuracy: 0.0053 - val_loss: 4.6270 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 30s 632ms/step - loss: 4.6327 - accuracy: 0.0108 - val_loss: 4.6283 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 30s 630ms/step - loss: 4.6331 - accuracy: 0.0098 - val_loss: 4.6283 - val_accuracy: 0.0088
Epoch 6/10
32/32 [==============================] - 29s 615ms/step - loss: 4.6267 - accuracy: 0.0078 - val_loss: 4.5481 - val_accuracy: 0.0118
Epoch 7/10
32/32 [==============================] - 29s 631ms/step - loss: 4.5398 - accuracy: 0.0173 - val_loss: 4.4553 - val_accuracy: 0.0216

Epoch 1/10
32/32 [==============================] - 40s 664ms/step - loss: 5.1354 - accuracy: 0.0064 - val_loss: 4.6534 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 29s 626ms/step - loss: 4.6654 - accuracy: 0.0035 - val_loss: 4.6424 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 29s 629ms/step - loss: 4.6428 - accuracy: 0.0054 - val_loss: 4.6313 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 30s 637ms/step - loss: 4.6457 - accuracy: 0.0079 - val_loss: 4.6271 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 30s 633ms/step - loss: 4.6378 - accuracy: 0.0075 - val_loss: 4.6221 - val_accuracy: 0.0127
Epoch 6/10
32/32 [==============================] - 30s 641ms/step - loss: 4.6205 - accuracy: 0.0117 - val_loss: 4.5119 - val_accuracy: 0.0186
Epoch 7/10
32/32 [==============================] - 29s 633ms/step - loss: 4.4958 - accuracy: 0.0177 - val_loss: 4.3952 - val_accuracy: 0.0255

In [24]:
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay75.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
32/32 [==============================] - 40s 661ms/step - loss: 5.9666 - accuracy: 0.0089 - val_loss: 4.6392 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 29s 623ms/step - loss: 4.6402 - accuracy: 0.0068 - val_loss: 4.6279 - val_accuracy: 0.0108
Epoch 3/10
32/32 [==============================] - 30s 636ms/step - loss: 4.6341 - accuracy: 0.0118 - val_loss: 4.6270 - val_accuracy: 0.0088
Epoch 4/10
32/32 [==============================] - 29s 641ms/step - loss: 4.6332 - accuracy: 0.0082 - val_loss: 4.6204 - val_accuracy: 0.0118
Epoch 5/10
32/32 [==============================] - 31s 660ms/step - loss: 4.6298 - accuracy: 0.0097 - val_loss: 4.6169 - val_accuracy: 0.0108
Epoch 6/10
32/32 [==============================] - 30s 645ms/step - loss: 4.6266 - accuracy: 0.0059 - val_loss: 4.6174 - val_accuracy: 0.0137
Epoch 7/10
32/32 [==============================] - 30s 640ms/step - loss: 4.6133 - accuracy: 0.0126 - val_loss: 4.5771 - val_accuracy: 0.0176

Epoch 1/10
32/32 [==============================] - 39s 662ms/step - loss: 6.0582 - accuracy: 0.0139 - val_loss: 4.6595 - val_accuracy: 0.0108
Epoch 2/10
32/32 [==============================] - 29s 610ms/step - loss: 4.6646 - accuracy: 0.0058 - val_loss: 4.6341 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 29s 611ms/step - loss: 4.6387 - accuracy: 0.0089 - val_loss: 4.6242 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 29s 614ms/step - loss: 4.6238 - accuracy: 0.0162 - val_loss: 4.6280 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 29s 623ms/step - loss: 4.6333 - accuracy: 0.0124 - val_loss: 4.6237 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 28s 612ms/step - loss: 4.6277 - accuracy: 0.0084 - val_loss: 4.5782 - val_accuracy: 0.0167
Epoch 7/10
32/32 [==============================] - 28s 601ms/step - loss: 4.5468 - accuracy: 0.0150 - val_loss: 4.3466 - val_accuracy: 0.0294

In [25]:
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
32/32 [==============================] - 40s 671ms/step - loss: 4.8553 - accuracy: 0.0063 - val_loss: 4.6253 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 30s 644ms/step - loss: 4.6255 - accuracy: 0.0124 - val_loss: 4.6252 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 29s 643ms/step - loss: 4.6256 - accuracy: 0.0078 - val_loss: 4.6252 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 30s 627ms/step - loss: 4.6248 - accuracy: 0.0093 - val_loss: 4.6252 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 30s 644ms/step - loss: 4.6253 - accuracy: 0.0126 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 30s 654ms/step - loss: 4.6250 - accuracy: 0.0156 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 29s 631ms/step - loss: 4.6256 - accuracy: 0.0112 - val_loss: 4.6251 - val_accuracy: 0.0098

Epoch 1/10
32/32 [==============================] - 40s 642ms/step - loss: 4.8565 - accuracy: 0.0163 - val_loss: 4.6252 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 29s 628ms/step - loss: 4.6261 - accuracy: 0.0102 - val_loss: 4.6252 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 29s 639ms/step - loss: 4.6252 - accuracy: 0.0081 - val_loss: 4.6252 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 29s 625ms/step - loss: 4.6246 - accuracy: 0.0088 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 29s 617ms/step - loss: 4.6252 - accuracy: 0.0059 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 30s 633ms/step - loss: 4.6258 - accuracy: 0.0073 - val_loss: 4.6251 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 30s 633ms/step - loss: 4.6256 - accuracy: 0.0107 - val_loss: 4.6251 - val_accuracy: 0.0098

In [26]:
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='flowers_models_optimized/EfficentNetB0_flowers_model_PolynomialDecay90.h5', 
                     ds_train=flowers_datasets[0], 
                     ds_validation=flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
32/32 [==============================] - 39s 663ms/step - loss: 5.8399 - accuracy: 0.0117 - val_loss: 4.6439 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 29s 627ms/step - loss: 4.6525 - accuracy: 0.0026 - val_loss: 4.6350 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 30s 641ms/step - loss: 4.6498 - accuracy: 0.0108 - val_loss: 4.6284 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 29s 609ms/step - loss: 4.6356 - accuracy: 0.0023 - val_loss: 4.6256 - val_accuracy: 0.0108
Epoch 5/10
32/32 [==============================] - 30s 625ms/step - loss: 4.6365 - accuracy: 0.0062 - val_loss: 4.6247 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 30s 652ms/step - loss: 4.6281 - accuracy: 0.0049 - val_loss: 4.6311 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 30s 648ms/step - loss: 4.6399 - accuracy: 0.0137 - val_loss: 4.6209 - val_accuracy: 0.0176

Epoch 1/10
32/32 [==============================] - 43s 685ms/step - loss: 5.8674 - accuracy: 0.0096 - val_loss: 4.6464 - val_accuracy: 0.0098
Epoch 2/10
32/32 [==============================] - 29s 633ms/step - loss: 4.6510 - accuracy: 0.0022 - val_loss: 4.6337 - val_accuracy: 0.0098
Epoch 3/10
32/32 [==============================] - 30s 639ms/step - loss: 4.6452 - accuracy: 0.0070 - val_loss: 4.6269 - val_accuracy: 0.0098
Epoch 4/10
32/32 [==============================] - 29s 616ms/step - loss: 4.6350 - accuracy: 0.0023 - val_loss: 4.6354 - val_accuracy: 0.0098
Epoch 5/10
32/32 [==============================] - 30s 636ms/step - loss: 4.6426 - accuracy: 0.0079 - val_loss: 4.6280 - val_accuracy: 0.0098
Epoch 6/10
32/32 [==============================] - 29s 622ms/step - loss: 4.6353 - accuracy: 0.0113 - val_loss: 4.6316 - val_accuracy: 0.0098
Epoch 7/10
32/32 [==============================] - 29s 618ms/step - loss: 4.6375 - accuracy: 0.0115 - val_loss: 4.6282 - val_accuracy: 0.0098

## 1. Prune base models

In [33]:
prune_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model.h5', 
            ds_train=tf_flowers_datasets[0], 
            ds_validation=tf_flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model.h5', 
            ds_train=tf_flowers_datasets[0], 
            ds_validation=tf_flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model.h5', 
            ds_train=tf_flowers_datasets[0], 
            ds_validation=tf_flowers_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

Epoch 1/10
 6/81 [=>............................] - ETA: 5:33 - loss: 4.5759 - accuracy: 0.1030WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1111s vs `on_train_batch_end` time: 3.5971s). Check your callbacks.


81/81 [==============================] - 71s 565ms/step - loss: 2.3130 - accuracy: 0.3017 - val_loss: 4.1403 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 41s 284ms/step - loss: 1.2401 - accuracy: 0.5021 - val_loss: 3.7335 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 36s 290ms/step - loss: 1.0454 - accuracy: 0.5865 - val_loss: 3.2857 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 42s 300ms/step - loss: 0.9488 - accuracy: 0.6282 - val_loss: 2.8430 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 40s 295ms/step - loss: 0.8219 - accuracy: 0.6928 - val_loss: 2.4836 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 36s 288ms/step - loss: 0.7694 - accuracy: 0.7190 - val_loss: 2.2601 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 34s 293ms/step - loss: 0.6924 - accuracy: 0.7573 - val_loss: 2.1318 - val_accuracy: 0.2668
Epoch 8/10

81/81 [==============================] - 63s 349ms/step - loss: 1.9113 - accuracy: 0.3597 - val_loss: 3.7687 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 37s 289ms/step - loss: 1.1633 - accuracy: 0.5210 - val_loss: 2.5998 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 34s 302ms/step - loss: 0.9927 - accuracy: 0.6043 - val_loss: 1.9434 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 42s 301ms/step - loss: 0.9659 - accuracy: 0.6386 - val_loss: 1.9169 - val_accuracy: 0.1379
Epoch 5/10
81/81 [==============================] - 41s 293ms/step - loss: 0.8366 - accuracy: 0.6795 - val_loss: 1.8433 - val_accuracy: 0.1379
Epoch 6/10
81/81 [==============================] - 37s 298ms/step - loss: 0.7373 - accuracy: 0.7146 - val_loss: 2.0342 - val_accuracy: 0.1379
Epoch 7/10
81/81 [==============================] - 30s 294ms/step - loss: 0.7201 - accuracy: 0.7264 - val_loss: 2.0955 - val_accuracy: 0.1379
Epoch 8/10

81/81 [==============================] - 68s 351ms/step - loss: 2.5821 - accuracy: 0.2989 - val_loss: 4.1028 - val_accuracy: 0.2432
Epoch 2/10
81/81 [==============================] - 40s 293ms/step - loss: 1.2174 - accuracy: 0.5222 - val_loss: 3.8983 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 34s 293ms/step - loss: 1.0064 - accuracy: 0.5994 - val_loss: 3.6618 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 30s 288ms/step - loss: 0.8323 - accuracy: 0.6793 - val_loss: 3.4097 - val_accuracy: 0.1379
Epoch 5/10
81/81 [==============================] - 41s 291ms/step - loss: 0.7839 - accuracy: 0.7006 - val_loss: 3.1322 - val_accuracy: 0.1379
Epoch 6/10
81/81 [==============================] - 39s 291ms/step - loss: 0.7233 - accuracy: 0.7395 - val_loss: 2.8117 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 42s 297ms/step - loss: 0.6966 - accuracy: 0.7342 - val_loss: 2.5622 - val_accuracy: 0.1379
Epoch 8/10

81/81 [==============================] - 62s 348ms/step - loss: 1.8941 - accuracy: 0.3613 - val_loss: 3.8100 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 36s 298ms/step - loss: 1.1538 - accuracy: 0.5417 - val_loss: 2.6510 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 41s 291ms/step - loss: 0.9935 - accuracy: 0.6045 - val_loss: 2.0047 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 33s 298ms/step - loss: 0.8201 - accuracy: 0.6854 - val_loss: 1.9132 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 41s 292ms/step - loss: 0.7697 - accuracy: 0.7195 - val_loss: 2.1138 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 33s 308ms/step - loss: 0.7354 - accuracy: 0.7287 - val_loss: 2.7425 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 42s 304ms/step - loss: 0.6434 - accuracy: 0.7601 - val_loss: 2.8300 - val_accuracy: 0.2668
Epoch 8/10

81/81 [==============================] - 68s 357ms/step - loss: 3.0089 - accuracy: 0.2821 - val_loss: 4.0149 - val_accuracy: 0.1379
Epoch 2/10
81/81 [==============================] - 42s 298ms/step - loss: 1.2222 - accuracy: 0.5244 - val_loss: 3.7721 - val_accuracy: 0.1379
Epoch 3/10
81/81 [==============================] - 30s 299ms/step - loss: 1.0686 - accuracy: 0.5764 - val_loss: 3.5780 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 36s 291ms/step - loss: 0.8916 - accuracy: 0.6710 - val_loss: 3.3956 - val_accuracy: 0.1379
Epoch 5/10
81/81 [==============================] - 40s 296ms/step - loss: 0.8846 - accuracy: 0.6572 - val_loss: 3.2167 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 42s 294ms/step - loss: 0.7832 - accuracy: 0.7145 - val_loss: 3.0371 - val_accuracy: 0.1379
Epoch 7/10
81/81 [==============================] - 29s 291ms/step - loss: 0.6908 - accuracy: 0.7372 - val_loss: 2.8535 - val_accuracy: 0.2668
Epoch 8/10

81/81 [==============================] - 60s 339ms/step - loss: 1.9276 - accuracy: 0.3228 - val_loss: 3.7759 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 42s 296ms/step - loss: 1.1623 - accuracy: 0.5348 - val_loss: 2.6165 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 29s 295ms/step - loss: 1.0136 - accuracy: 0.5993 - val_loss: 1.9715 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 41s 290ms/step - loss: 0.8576 - accuracy: 0.6775 - val_loss: 1.8582 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 40s 296ms/step - loss: 0.8442 - accuracy: 0.6910 - val_loss: 1.8179 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 37s 293ms/step - loss: 0.8555 - accuracy: 0.6656 - val_loss: 1.9323 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 41s 292ms/step - loss: 0.6538 - accuracy: 0.7600 - val_loss: 1.8620 - val_accuracy: 0.2668
Epoch 8/10

In [34]:
prune_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model.h5', 
            ds_train=tf_flowers_datasets[0], 
            ds_validation=tf_flowers_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model.h5', 
            ds_train=tf_flowers_datasets[0], 
            ds_validation=tf_flowers_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model.h5', 
            ds_train=tf_flowers_datasets[0], 
            ds_validation=tf_flowers_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

Epoch 1/10
 6/81 [=>............................] - ETA: 1:28 - loss: 4.5947 - accuracy: 0.1095WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1698s vs `on_train_batch_end` time: 0.8223s). Check your callbacks.


81/81 [==============================] - 91s 495ms/step - loss: 2.7770 - accuracy: 0.3115 - val_loss: 2.1217 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 49s 427ms/step - loss: 1.2693 - accuracy: 0.4822 - val_loss: 2.2240 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 48s 433ms/step - loss: 1.0239 - accuracy: 0.5720 - val_loss: 3.0491 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 46s 427ms/step - loss: 0.8934 - accuracy: 0.6448 - val_loss: 2.5905 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 49s 437ms/step - loss: 0.8031 - accuracy: 0.6904 - val_loss: 3.8008 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 49s 429ms/step - loss: 0.6531 - accuracy: 0.7499 - val_loss: 4.6803 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 48s 437ms/step - loss: 0.5574 - accuracy: 0.7845 - val_loss: 3.5511 - val_accuracy: 0.2668
Epoch 8/10

81/81 [==============================] - 82s 487ms/step - loss: 2.4952 - accuracy: 0.3514 - val_loss: 2.0025 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 43s 433ms/step - loss: 1.1718 - accuracy: 0.5407 - val_loss: 2.6527 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 50s 426ms/step - loss: 1.0170 - accuracy: 0.5955 - val_loss: 4.9386 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 46s 431ms/step - loss: 0.8469 - accuracy: 0.6704 - val_loss: 3.3533 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 44s 433ms/step - loss: 0.7527 - accuracy: 0.7187 - val_loss: 4.9370 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 52s 427ms/step - loss: 0.6247 - accuracy: 0.7602 - val_loss: 4.3971 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 49s 425ms/step - loss: 0.6468 - accuracy: 0.7640 - val_loss: 6.2506 - val_accuracy: 0.2704
Epoch 8/10

81/81 [==============================] - 91s 498ms/step - loss: 3.0527 - accuracy: 0.3133 - val_loss: 2.5551 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 46s 431ms/step - loss: 1.2297 - accuracy: 0.4979 - val_loss: 2.0305 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 50s 425ms/step - loss: 1.0237 - accuracy: 0.5678 - val_loss: 2.2160 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 50s 424ms/step - loss: 0.8997 - accuracy: 0.6384 - val_loss: 2.6019 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 46s 429ms/step - loss: 0.7816 - accuracy: 0.6955 - val_loss: 2.6816 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 52s 425ms/step - loss: 0.6694 - accuracy: 0.7556 - val_loss: 2.7700 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 49s 429ms/step - loss: 0.5268 - accuracy: 0.8246 - val_loss: 3.0881 - val_accuracy: 0.2650
Epoch 8/10

81/81 [==============================] - 81s 482ms/step - loss: 2.5172 - accuracy: 0.3296 - val_loss: 2.1950 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 42s 428ms/step - loss: 1.1954 - accuracy: 0.5263 - val_loss: 3.1524 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 46s 426ms/step - loss: 1.0475 - accuracy: 0.5849 - val_loss: 2.1915 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 52s 433ms/step - loss: 0.9338 - accuracy: 0.6585 - val_loss: 3.0729 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 44s 434ms/step - loss: 0.7643 - accuracy: 0.7150 - val_loss: 3.0235 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 51s 424ms/step - loss: 0.6383 - accuracy: 0.7651 - val_loss: 3.9738 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 52s 431ms/step - loss: 0.5427 - accuracy: 0.8056 - val_loss: 6.5326 - val_accuracy: 0.2668
Epoch 8/10

81/81 [==============================] - 83s 500ms/step - loss: 3.7153 - accuracy: 0.2564 - val_loss: 3.4246 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 43s 435ms/step - loss: 1.3920 - accuracy: 0.5158 - val_loss: 2.2642 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 52s 429ms/step - loss: 1.0527 - accuracy: 0.6054 - val_loss: 1.9383 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 52s 431ms/step - loss: 0.9078 - accuracy: 0.6538 - val_loss: 2.3867 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 50s 433ms/step - loss: 0.7906 - accuracy: 0.7017 - val_loss: 1.9353 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 52s 432ms/step - loss: 0.6716 - accuracy: 0.7445 - val_loss: 1.8627 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 52s 424ms/step - loss: 0.6203 - accuracy: 0.7705 - val_loss: 3.3683 - val_accuracy: 0.2668
Epoch 8/10

81/81 [==============================] - 85s 484ms/step - loss: 2.4429 - accuracy: 0.3120 - val_loss: 2.6988 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 48s 426ms/step - loss: 1.2301 - accuracy: 0.5080 - val_loss: 3.1534 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 51s 429ms/step - loss: 1.0613 - accuracy: 0.5878 - val_loss: 6.4755 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 47s 425ms/step - loss: 0.9271 - accuracy: 0.6260 - val_loss: 2.4142 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 40s 432ms/step - loss: 0.7321 - accuracy: 0.7318 - val_loss: 3.1565 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 52s 429ms/step - loss: 0.6466 - accuracy: 0.7524 - val_loss: 3.1928 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 52s 436ms/step - loss: 0.4857 - accuracy: 0.8388 - val_loss: 5.6739 - val_accuracy: 0.2704
Epoch 8/10

In [35]:
prune_model(model_path='beans_models/MobileNetV2_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='beans_models/MobileNetV2_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='beans_models/MobileNetV2_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1],
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

Epoch 1/10
 6/33 [====>.........................] - ETA: 43s - loss: 1.2582 - accuracy: 0.4089WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.2328s vs `on_train_batch_end` time: 1.1799s). Check your callbacks.


33/33 [==============================] - 63s 1s/step - loss: 1.0580 - accuracy: 0.5242 - val_loss: 1.0987 - val_accuracy: 0.3383
Epoch 2/10
33/33 [==============================] - 34s 944ms/step - loss: 0.7599 - accuracy: 0.6544 - val_loss: 1.0988 - val_accuracy: 0.3383
Epoch 3/10
33/33 [==============================] - 33s 938ms/step - loss: 0.6567 - accuracy: 0.7104 - val_loss: 1.0987 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 33s 934ms/step - loss: 0.5975 - accuracy: 0.7415 - val_loss: 1.1006 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 34s 943ms/step - loss: 0.6047 - accuracy: 0.7501 - val_loss: 1.1056 - val_accuracy: 0.3383
Epoch 6/10
33/33 [==============================] - 33s 931ms/step - loss: 0.5094 - accuracy: 0.7799 - val_loss: 1.1148 - val_accuracy: 0.3383
Epoch 7/10
33/33 [==============================] - 34s 942ms/step - loss: 0.4857 - accuracy: 0.7898 - val_loss: 1.1255 - val_accuracy: 0.3383
Epoch 8/10
33

33/33 [==============================] - 58s 1s/step - loss: 1.1375 - accuracy: 0.4923 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 2/10
33/33 [==============================] - 34s 946ms/step - loss: 0.7364 - accuracy: 0.6711 - val_loss: 1.1004 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 34s 945ms/step - loss: 0.6044 - accuracy: 0.7447 - val_loss: 1.1034 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 34s 940ms/step - loss: 0.6225 - accuracy: 0.7319 - val_loss: 1.1139 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 34s 949ms/step - loss: 0.5036 - accuracy: 0.7940 - val_loss: 1.1235 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 34s 949ms/step - loss: 0.5373 - accuracy: 0.7733 - val_loss: 1.1259 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 34s 940ms/step - loss: 0.4764 - accuracy: 0.7959 - val_loss: 1.1723 - val_accuracy: 0.3308
Epoch 8/10
33

33/33 [==============================] - 57s 1s/step - loss: 1.0942 - accuracy: 0.4262 - val_loss: 1.0988 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 34s 940ms/step - loss: 0.8000 - accuracy: 0.6357 - val_loss: 1.0997 - val_accuracy: 0.3383
Epoch 3/10
33/33 [==============================] - 34s 939ms/step - loss: 0.6730 - accuracy: 0.7069 - val_loss: 1.1002 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 34s 946ms/step - loss: 0.6462 - accuracy: 0.7199 - val_loss: 1.1023 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 33s 931ms/step - loss: 0.5160 - accuracy: 0.8032 - val_loss: 1.1072 - val_accuracy: 0.3383
Epoch 6/10
33/33 [==============================] - 34s 937ms/step - loss: 0.4331 - accuracy: 0.8209 - val_loss: 1.1127 - val_accuracy: 0.3383
Epoch 7/10
33/33 [==============================] - 34s 948ms/step - loss: 0.4080 - accuracy: 0.8276 - val_loss: 1.1202 - val_accuracy: 0.3383
Epoch 8/10
33

33/33 [==============================] - 57s 1s/step - loss: 1.0911 - accuracy: 0.5220 - val_loss: 1.0987 - val_accuracy: 0.3383
Epoch 2/10
33/33 [==============================] - 33s 936ms/step - loss: 0.8003 - accuracy: 0.6502 - val_loss: 1.1039 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 34s 946ms/step - loss: 0.6861 - accuracy: 0.6927 - val_loss: 1.1006 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 33s 937ms/step - loss: 0.6001 - accuracy: 0.7363 - val_loss: 1.1432 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 33s 930ms/step - loss: 0.6631 - accuracy: 0.6861 - val_loss: 1.1556 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 34s 941ms/step - loss: 0.5748 - accuracy: 0.7750 - val_loss: 1.1852 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 33s 930ms/step - loss: 0.5879 - accuracy: 0.7825 - val_loss: 1.2715 - val_accuracy: 0.3308
Epoch 8/10
33

33/33 [==============================] - 61s 1s/step - loss: 0.9871 - accuracy: 0.4844 - val_loss: 1.0987 - val_accuracy: 0.3383
Epoch 2/10
33/33 [==============================] - 33s 940ms/step - loss: 0.7168 - accuracy: 0.6863 - val_loss: 1.0987 - val_accuracy: 0.3383
Epoch 3/10
33/33 [==============================] - 34s 957ms/step - loss: 0.6220 - accuracy: 0.7447 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 34s 957ms/step - loss: 0.5946 - accuracy: 0.7379 - val_loss: 1.0987 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 33s 934ms/step - loss: 0.4774 - accuracy: 0.8118 - val_loss: 1.0988 - val_accuracy: 0.3383
Epoch 6/10
33/33 [==============================] - 34s 957ms/step - loss: 0.5079 - accuracy: 0.7952 - val_loss: 1.0988 - val_accuracy: 0.3383
Epoch 7/10
33/33 [==============================] - 34s 945ms/step - loss: 0.4846 - accuracy: 0.7992 - val_loss: 1.0987 - val_accuracy: 0.3383
Epoch 8/10
33

33/33 [==============================] - 57s 1s/step - loss: 1.0802 - accuracy: 0.5095 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 34s 943ms/step - loss: 0.7709 - accuracy: 0.6795 - val_loss: 1.0987 - val_accuracy: 0.3383
Epoch 3/10
33/33 [==============================] - 34s 945ms/step - loss: 0.6495 - accuracy: 0.7127 - val_loss: 1.1030 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 34s 944ms/step - loss: 0.6638 - accuracy: 0.7155 - val_loss: 1.1159 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 34s 946ms/step - loss: 0.5686 - accuracy: 0.7761 - val_loss: 1.1148 - val_accuracy: 0.3383
Epoch 6/10
33/33 [==============================] - 34s 939ms/step - loss: 0.5769 - accuracy: 0.7559 - val_loss: 1.1288 - val_accuracy: 0.3383
Epoch 7/10
33/33 [==============================] - 34s 949ms/step - loss: 0.4448 - accuracy: 0.8215 - val_loss: 1.1583 - val_accuracy: 0.3383
Epoch 8/10
33

In [36]:
prune_model(model_path='beans_models/EfficentNetB0_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.5)
prune_model(model_path='beans_models/EfficentNetB0_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.75)
prune_model(model_path='beans_models/EfficentNetB0_beans_model.h5', 
            ds_train=beans_datasets[0], 
            ds_validation=beans_datasets[1], 
            batch_size=BATCH_SIZE, 
            pruning_epochs=PRUNING_EPOCHS,
            sparsity=0.9)

Epoch 1/10
 6/33 [====>.........................] - ETA: 57s - loss: 1.1102 - accuracy: 0.4663 WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.4026s vs `on_train_batch_end` time: 1.4952s). Check your callbacks.


33/33 [==============================] - 83s 2s/step - loss: 1.0568 - accuracy: 0.4857 - val_loss: 1.1230 - val_accuracy: 0.3383
Epoch 2/10
33/33 [==============================] - 47s 1s/step - loss: 0.8465 - accuracy: 0.6143 - val_loss: 1.1769 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 47s 1s/step - loss: 0.7222 - accuracy: 0.6713 - val_loss: 1.4131 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 0.6649 - accuracy: 0.7252 - val_loss: 2.6931 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 0.5331 - accuracy: 0.7888 - val_loss: 4.3021 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 0.6056 - accuracy: 0.7689 - val_loss: 3.2661 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 0.5269 - accuracy: 0.7529 - val_loss: 4.3162 - val_accuracy: 0.3308
Epoch 8/10
33/33 [=============

33/33 [==============================] - 83s 2s/step - loss: 1.2533 - accuracy: 0.4542 - val_loss: 1.1948 - val_accuracy: 0.3383
Epoch 2/10
33/33 [==============================] - 46s 1s/step - loss: 0.8474 - accuracy: 0.5998 - val_loss: 1.5107 - val_accuracy: 0.3383
Epoch 3/10
33/33 [==============================] - 47s 1s/step - loss: 0.7126 - accuracy: 0.6949 - val_loss: 2.0215 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 0.6479 - accuracy: 0.7326 - val_loss: 3.4888 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 46s 1s/step - loss: 0.6512 - accuracy: 0.7431 - val_loss: 2.8643 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 46s 1s/step - loss: 0.5423 - accuracy: 0.7941 - val_loss: 2.1348 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 0.4495 - accuracy: 0.8123 - val_loss: 2.2361 - val_accuracy: 0.3308
Epoch 8/10
33/33 [=============

33/33 [==============================] - 89s 2s/step - loss: 1.1213 - accuracy: 0.4165 - val_loss: 1.1008 - val_accuracy: 0.3383
Epoch 2/10
33/33 [==============================] - 47s 1s/step - loss: 0.8677 - accuracy: 0.6247 - val_loss: 1.1149 - val_accuracy: 0.3383
Epoch 3/10
33/33 [==============================] - 47s 1s/step - loss: 0.7894 - accuracy: 0.6511 - val_loss: 1.2113 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 0.6255 - accuracy: 0.7341 - val_loss: 1.3920 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 0.5421 - accuracy: 0.7644 - val_loss: 1.5689 - val_accuracy: 0.3383
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 0.4473 - accuracy: 0.8343 - val_loss: 1.7158 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 0.4750 - accuracy: 0.8112 - val_loss: 1.9140 - val_accuracy: 0.3308
Epoch 8/10
33/33 [=============

33/33 [==============================] - 86s 2s/step - loss: 1.3346 - accuracy: 0.4353 - val_loss: 1.2988 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 47s 1s/step - loss: 0.8214 - accuracy: 0.6472 - val_loss: 1.5132 - val_accuracy: 0.3383
Epoch 3/10
33/33 [==============================] - 47s 1s/step - loss: 0.7280 - accuracy: 0.6780 - val_loss: 2.5111 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 0.7389 - accuracy: 0.6962 - val_loss: 2.2731 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 0.6351 - accuracy: 0.7608 - val_loss: 4.3877 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 0.5161 - accuracy: 0.7825 - val_loss: 2.8250 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 0.5178 - accuracy: 0.7840 - val_loss: 3.9455 - val_accuracy: 0.3308
Epoch 8/10
33/33 [=============

33/33 [==============================] - 87s 2s/step - loss: 1.0806 - accuracy: 0.4261 - val_loss: 1.1002 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 47s 1s/step - loss: 0.9132 - accuracy: 0.5807 - val_loss: 1.1098 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 47s 1s/step - loss: 0.7824 - accuracy: 0.6558 - val_loss: 1.1399 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 0.6814 - accuracy: 0.7172 - val_loss: 1.1701 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 0.6319 - accuracy: 0.7276 - val_loss: 1.2646 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 0.5544 - accuracy: 0.7857 - val_loss: 1.4420 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 0.4697 - accuracy: 0.8215 - val_loss: 1.5343 - val_accuracy: 0.3308
Epoch 8/10
33/33 [=============

33/33 [==============================] - 86s 2s/step - loss: 1.3584 - accuracy: 0.4286 - val_loss: 1.2842 - val_accuracy: 0.3383
Epoch 2/10
33/33 [==============================] - 47s 1s/step - loss: 0.9221 - accuracy: 0.6298 - val_loss: 2.0954 - val_accuracy: 0.3383
Epoch 3/10
33/33 [==============================] - 47s 1s/step - loss: 0.7141 - accuracy: 0.7238 - val_loss: 1.3336 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 0.6551 - accuracy: 0.7343 - val_loss: 1.7497 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 0.6011 - accuracy: 0.7753 - val_loss: 1.7407 - val_accuracy: 0.3383
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 0.5365 - accuracy: 0.7868 - val_loss: 1.7450 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 0.5451 - accuracy: 0.7839 - val_loss: 2.3441 - val_accuracy: 0.3308
Epoch 8/10
33/33 [=============

## 2. Weight cluster base models

In [27]:
weight_cluster_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
81/81 [==============================] - 44s 289ms/step - loss: 0.2748 - accuracy: 0.9220 - val_loss: 1.7442 - val_accuracy: 0.6733
Epoch 2/10
81/81 [==============================] - 31s 267ms/step - loss: 0.0366 - accuracy: 0.9874 - val_loss: 2.1898 - val_accuracy: 0.6352
Epoch 3/10
81/81 [==============================] - 38s 268ms/step - loss: 0.0765 - accuracy: 0.9735 - val_loss: 1.5476 - val_accuracy: 0.6606
Epoch 4/10
81/81 [==============================] - 39s 262ms/step - loss: 0.0739 - accuracy: 0.9722 - val_loss: 1.6050 - val_accuracy: 0.6552
Epoch 5/10
81/81 [==============================] - 37s 269ms/step - loss: 0.0437 - accuracy: 0.9884 - val_loss: 1.6673 - val_accuracy: 0.6461
Epoch 6/10
81/81 [==============================] - 30s 270ms/step - loss: 0.0325 - accuracy: 0.9897 - val_loss: 1.9145 - val_accuracy: 0.6661
Epoch 7/10
81/81 [==============================] - 40s 274ms/step - loss: 0.0582 - accuracy: 0.9796 - val_loss: 1.9791 - val_accuracy: 0.6751

In [28]:
weight_cluster_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
81/81 [==============================] - 59s 407ms/step - loss: 0.0385 - accuracy: 0.9884 - val_loss: 1.8030 - val_accuracy: 0.7078
Epoch 2/10
81/81 [==============================] - 45s 385ms/step - loss: 0.0061 - accuracy: 0.9976 - val_loss: 1.5411 - val_accuracy: 0.6425
Epoch 3/10
81/81 [==============================] - 47s 381ms/step - loss: 0.0236 - accuracy: 0.9937 - val_loss: 1.9565 - val_accuracy: 0.6987
Epoch 4/10
81/81 [==============================] - 47s 379ms/step - loss: 0.0321 - accuracy: 0.9880 - val_loss: 1.7340 - val_accuracy: 0.6842
Epoch 5/10
81/81 [==============================] - 49s 378ms/step - loss: 0.0027 - accuracy: 0.9991 - val_loss: 2.3304 - val_accuracy: 0.6697
Epoch 6/10
81/81 [==============================] - 45s 385ms/step - loss: 0.0335 - accuracy: 0.9880 - val_loss: 2.0664 - val_accuracy: 0.6933
Epoch 7/10
81/81 [==============================] - 38s 381ms/step - loss: 0.0233 - accuracy: 0.9894 - val_loss: 1.8362 - val_accuracy: 0.6552

In [29]:
weight_cluster_model(model_path='beans_models/MobileNetV2_beans_model.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models/MobileNetV2_beans_model.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
33/33 [==============================] - 44s 1s/step - loss: 3.7637 - accuracy: 0.6958 - val_loss: 0.7142 - val_accuracy: 0.6541
Epoch 2/10
33/33 [==============================] - 36s 986ms/step - loss: 0.6046 - accuracy: 0.7246 - val_loss: 0.7573 - val_accuracy: 0.6241
Epoch 3/10
33/33 [==============================] - 36s 1s/step - loss: 0.4702 - accuracy: 0.8134 - val_loss: 0.5811 - val_accuracy: 0.7519
Epoch 4/10
33/33 [==============================] - 36s 1s/step - loss: 0.2608 - accuracy: 0.9114 - val_loss: 0.9700 - val_accuracy: 0.6767
Epoch 5/10
33/33 [==============================] - 37s 1s/step - loss: 0.2134 - accuracy: 0.9278 - val_loss: 0.7754 - val_accuracy: 0.7368
Epoch 6/10
33/33 [==============================] - 36s 1s/step - loss: 0.2293 - accuracy: 0.8988 - val_loss: 0.9504 - val_accuracy: 0.6767
Epoch 7/10
33/33 [==============================] - 37s 1s/step - loss: 0.0676 - accuracy: 0.9762 - val_loss: 1.2316 - val_accuracy: 0.7669
Epoch 8/10
33/33 

In [31]:
weight_cluster_model(model_path='beans_models/EfficentNetB0_beans_model.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models/EfficentNetB0_beans_model.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
33/33 [==============================] - 58s 1s/step - loss: 0.0369 - accuracy: 0.9912 - val_loss: 1.3306 - val_accuracy: 0.6842
Epoch 2/10
33/33 [==============================] - 47s 1s/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 1.3111 - val_accuracy: 0.7444
Epoch 3/10
33/33 [==============================] - 47s 1s/step - loss: 5.6424e-05 - accuracy: 1.0000 - val_loss: 1.3342 - val_accuracy: 0.7519
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 1.6206e-05 - accuracy: 1.0000 - val_loss: 1.3479 - val_accuracy: 0.7519
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 9.0645e-06 - accuracy: 1.0000 - val_loss: 1.3593 - val_accuracy: 0.7519
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 1.1409e-05 - accuracy: 1.0000 - val_loss: 1.3722 - val_accuracy: 0.7519
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 5.9880e-06 - accuracy: 1.0000 - val_loss: 1.3825 - val_accuracy: 0.7519


## 3. weight cluster prunned models

In [38]:
weight_cluster_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model_ConstantSparsity50.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model_ConstantSparsity50.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
81/81 [==============================] - 38s 284ms/step - loss: 1.7164 - accuracy: 0.2318 - val_loss: 1.5979 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 39s 280ms/step - loss: 1.6089 - accuracy: 0.2327 - val_loss: 1.6063 - val_accuracy: 0.1833
Epoch 3/10
81/81 [==============================] - 36s 276ms/step - loss: 1.6045 - accuracy: 0.2421 - val_loss: 1.6094 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 36s 285ms/step - loss: 1.6041 - accuracy: 0.2564 - val_loss: 1.6021 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 31s 286ms/step - loss: 1.6067 - accuracy: 0.2435 - val_loss: 1.5856 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 32s 282ms/step - loss: 1.6044 - accuracy: 0.2469 - val_loss: 1.5947 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 37s 275ms/step - loss: 1.6045 - accuracy: 0.2590 - val_loss: 1.5892 - val_accuracy: 0.2668

Epoch 1/10
81/81 [==============================] - 42s 284ms/step - loss: 1.6883 - accuracy: 0.2583 - val_loss: 1.5861 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 35s 272ms/step - loss: 1.6067 - accuracy: 0.2514 - val_loss: 1.5972 - val_accuracy: 0.2432
Epoch 3/10
81/81 [==============================] - 31s 279ms/step - loss: 1.6103 - accuracy: 0.2271 - val_loss: 1.5912 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 40s 275ms/step - loss: 1.6025 - accuracy: 0.2412 - val_loss: 1.5979 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 35s 275ms/step - loss: 1.6044 - accuracy: 0.2535 - val_loss: 1.5934 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 31s 276ms/step - loss: 1.6000 - accuracy: 0.2680 - val_loss: 1.6050 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 35s 277ms/step - loss: 1.6079 - accuracy: 0.2292 - val_loss: 1.5910 - val_accuracy: 0.2668

In [39]:
weight_cluster_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model_PolynomialDecay50.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model_PolynomialDecay50.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
81/81 [==============================] - 51s 361ms/step - loss: 1.8402 - accuracy: 0.2026 - val_loss: 1.5975 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 32s 293ms/step - loss: 1.6103 - accuracy: 0.2444 - val_loss: 1.5996 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 40s 279ms/step - loss: 1.6047 - accuracy: 0.2588 - val_loss: 1.5829 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 36s 285ms/step - loss: 1.6116 - accuracy: 0.2312 - val_loss: 1.5915 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 41s 281ms/step - loss: 1.6060 - accuracy: 0.2402 - val_loss: 1.5896 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 28s 282ms/step - loss: 1.6052 - accuracy: 0.2544 - val_loss: 1.5916 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 37s 282ms/step - loss: 1.6010 - accuracy: 0.2440 - val_loss: 1.6079 - val_accuracy: 0.2668

Epoch 1/10
81/81 [==============================] - 45s 284ms/step - loss: 1.8371 - accuracy: 0.2196 - val_loss: 1.5880 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 32s 274ms/step - loss: 1.6106 - accuracy: 0.2504 - val_loss: 1.5911 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 34s 276ms/step - loss: 1.6093 - accuracy: 0.2384 - val_loss: 1.6001 - val_accuracy: 0.2432
Epoch 4/10
81/81 [==============================] - 32s 272ms/step - loss: 1.6108 - accuracy: 0.2299 - val_loss: 1.6007 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 39s 269ms/step - loss: 1.6089 - accuracy: 0.2203 - val_loss: 1.6006 - val_accuracy: 0.1833
Epoch 6/10
81/81 [==============================] - 31s 267ms/step - loss: 1.6114 - accuracy: 0.2226 - val_loss: 1.5893 - val_accuracy: 0.2432
Epoch 7/10
81/81 [==============================] - 40s 276ms/step - loss: 1.6139 - accuracy: 0.2123 - val_loss: 1.5913 - val_accuracy: 0.2668

In [40]:
weight_cluster_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model_ConstantSparsity75.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model_ConstantSparsity75.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
81/81 [==============================] - 37s 285ms/step - loss: 1.7226 - accuracy: 0.2214 - val_loss: 1.5925 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 37s 282ms/step - loss: 1.6044 - accuracy: 0.2675 - val_loss: 1.5855 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 29s 281ms/step - loss: 1.6103 - accuracy: 0.2380 - val_loss: 1.5935 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 40s 283ms/step - loss: 1.6097 - accuracy: 0.2340 - val_loss: 1.5900 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 36s 278ms/step - loss: 1.5969 - accuracy: 0.2609 - val_loss: 1.6001 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 40s 279ms/step - loss: 1.6013 - accuracy: 0.2582 - val_loss: 1.5956 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 36s 284ms/step - loss: 1.6067 - accuracy: 0.2406 - val_loss: 1.5915 - val_accuracy: 0.2668

Epoch 1/10
81/81 [==============================] - 44s 296ms/step - loss: 1.7291 - accuracy: 0.2101 - val_loss: 1.6031 - val_accuracy: 0.1833
Epoch 2/10
81/81 [==============================] - 33s 281ms/step - loss: 1.6061 - accuracy: 0.2286 - val_loss: 1.5892 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 37s 279ms/step - loss: 1.6069 - accuracy: 0.2488 - val_loss: 1.6008 - val_accuracy: 0.2432
Epoch 4/10
81/81 [==============================] - 39s 282ms/step - loss: 1.6065 - accuracy: 0.2261 - val_loss: 1.5979 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 28s 282ms/step - loss: 1.5994 - accuracy: 0.2598 - val_loss: 1.5962 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 33s 282ms/step - loss: 1.6053 - accuracy: 0.2433 - val_loss: 1.5872 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 34s 282ms/step - loss: 1.6012 - accuracy: 0.2483 - val_loss: 1.5985 - val_accuracy: 0.2668

In [41]:
weight_cluster_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model_PolynomialDecay75.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model_PolynomialDecay75.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
81/81 [==============================] - 47s 302ms/step - loss: 2.5659 - accuracy: 0.2385 - val_loss: 1.5903 - val_accuracy: 0.2432
Epoch 2/10
81/81 [==============================] - 38s 285ms/step - loss: 1.6110 - accuracy: 0.2248 - val_loss: 1.6047 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 40s 282ms/step - loss: 1.6120 - accuracy: 0.2407 - val_loss: 1.5877 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 40s 282ms/step - loss: 1.6086 - accuracy: 0.2399 - val_loss: 1.5866 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 41s 288ms/step - loss: 1.6082 - accuracy: 0.2396 - val_loss: 1.5895 - val_accuracy: 0.2432
Epoch 6/10
81/81 [==============================] - 33s 283ms/step - loss: 1.6014 - accuracy: 0.2386 - val_loss: 1.5919 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 35s 285ms/step - loss: 1.6079 - accuracy: 0.2418 - val_loss: 1.5866 - val_accuracy: 0.2668

Epoch 1/10
81/81 [==============================] - 43s 299ms/step - loss: 2.4906 - accuracy: 0.2302 - val_loss: 1.5871 - val_accuracy: 0.2432
Epoch 2/10
81/81 [==============================] - 40s 286ms/step - loss: 1.6114 - accuracy: 0.2289 - val_loss: 1.6014 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 40s 285ms/step - loss: 1.6090 - accuracy: 0.2449 - val_loss: 1.5963 - val_accuracy: 0.2432
Epoch 4/10
81/81 [==============================] - 40s 289ms/step - loss: 1.6048 - accuracy: 0.2398 - val_loss: 1.6136 - val_accuracy: 0.1833
Epoch 5/10
81/81 [==============================] - 41s 282ms/step - loss: 1.6096 - accuracy: 0.2230 - val_loss: 1.5930 - val_accuracy: 0.2432
Epoch 6/10
81/81 [==============================] - 35s 287ms/step - loss: 1.6061 - accuracy: 0.2431 - val_loss: 1.6038 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 34s 282ms/step - loss: 1.6034 - accuracy: 0.2469 - val_loss: 1.5903 - val_accuracy: 0.2668

In [42]:
weight_cluster_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model_ConstantSparsity90.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model_ConstantSparsity90.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
81/81 [==============================] - 36s 297ms/step - loss: 1.7790 - accuracy: 0.2459 - val_loss: 1.5949 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 40s 285ms/step - loss: 1.6037 - accuracy: 0.2442 - val_loss: 1.5840 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 41s 290ms/step - loss: 1.5999 - accuracy: 0.2575 - val_loss: 1.5953 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 37s 281ms/step - loss: 1.6034 - accuracy: 0.2421 - val_loss: 1.5976 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 36s 286ms/step - loss: 1.6035 - accuracy: 0.2500 - val_loss: 1.6026 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 31s 283ms/step - loss: 1.6053 - accuracy: 0.2487 - val_loss: 1.5874 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 40s 283ms/step - loss: 1.6014 - accuracy: 0.2397 - val_loss: 1.5957 - val_accuracy: 0.2668

Epoch 1/10
81/81 [==============================] - 46s 290ms/step - loss: 1.7809 - accuracy: 0.2234 - val_loss: 1.5942 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 32s 283ms/step - loss: 1.6086 - accuracy: 0.2518 - val_loss: 1.5911 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 33s 277ms/step - loss: 1.6035 - accuracy: 0.2449 - val_loss: 1.5948 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 39s 272ms/step - loss: 1.6048 - accuracy: 0.2462 - val_loss: 1.5901 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 36s 275ms/step - loss: 1.6106 - accuracy: 0.2330 - val_loss: 1.5890 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 40s 279ms/step - loss: 1.6033 - accuracy: 0.2557 - val_loss: 1.5935 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 40s 278ms/step - loss: 1.6041 - accuracy: 0.2523 - val_loss: 1.5904 - val_accuracy: 0.2668

In [43]:
weight_cluster_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model_PolynomialDecay90.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='tf_flowers_models/MobileNetV2_tf_flowers_model_PolynomialDecay90.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
81/81 [==============================] - 46s 282ms/step - loss: 1.7853 - accuracy: 0.2300 - val_loss: 1.6153 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 37s 274ms/step - loss: 1.6056 - accuracy: 0.2670 - val_loss: 1.5926 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 39s 280ms/step - loss: 1.6039 - accuracy: 0.2494 - val_loss: 1.5937 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 40s 277ms/step - loss: 1.6061 - accuracy: 0.2414 - val_loss: 1.5958 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 40s 278ms/step - loss: 1.6079 - accuracy: 0.2498 - val_loss: 1.5886 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 38s 273ms/step - loss: 1.6035 - accuracy: 0.2469 - val_loss: 1.5963 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 39s 271ms/step - loss: 1.6045 - accuracy: 0.2470 - val_loss: 1.5900 - val_accuracy: 0.2668

Epoch 1/10
81/81 [==============================] - 50s 292ms/step - loss: 1.7580 - accuracy: 0.2154 - val_loss: 1.5874 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 37s 280ms/step - loss: 1.6056 - accuracy: 0.2250 - val_loss: 1.5982 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 40s 276ms/step - loss: 1.6072 - accuracy: 0.2371 - val_loss: 1.5910 - val_accuracy: 0.2668
Epoch 4/10
81/81 [==============================] - 32s 278ms/step - loss: 1.6052 - accuracy: 0.2501 - val_loss: 1.5954 - val_accuracy: 0.2668
Epoch 5/10
81/81 [==============================] - 32s 287ms/step - loss: 1.6070 - accuracy: 0.2314 - val_loss: 1.5985 - val_accuracy: 0.2668
Epoch 6/10
81/81 [==============================] - 28s 281ms/step - loss: 1.6072 - accuracy: 0.2317 - val_loss: 1.5914 - val_accuracy: 0.2668
Epoch 7/10
81/81 [==============================] - 35s 278ms/step - loss: 1.6078 - accuracy: 0.2489 - val_loss: 1.5898 - val_accuracy: 0.2668

In [44]:
weight_cluster_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model_ConstantSparsity50.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model_ConstantSparsity50.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
81/81 [==============================] - 58s 409ms/step - loss: 0.3119 - accuracy: 0.8851 - val_loss: 1.3623 - val_accuracy: 0.6824
Epoch 2/10
81/81 [==============================] - 49s 394ms/step - loss: 0.0748 - accuracy: 0.9774 - val_loss: 1.6688 - val_accuracy: 0.6316
Epoch 3/10
81/81 [==============================] - 47s 399ms/step - loss: 0.0794 - accuracy: 0.9753 - val_loss: 1.2669 - val_accuracy: 0.6933
Epoch 4/10
81/81 [==============================] - 44s 394ms/step - loss: 0.0413 - accuracy: 0.9894 - val_loss: 1.6762 - val_accuracy: 0.6624
Epoch 5/10
81/81 [==============================] - 48s 392ms/step - loss: 0.0330 - accuracy: 0.9892 - val_loss: 2.1733 - val_accuracy: 0.6751
Epoch 6/10
81/81 [==============================] - 49s 395ms/step - loss: 0.0614 - accuracy: 0.9816 - val_loss: 2.0821 - val_accuracy: 0.6860
Epoch 7/10
81/81 [==============================] - 48s 392ms/step - loss: 0.0239 - accuracy: 0.9912 - val_loss: 1.9853 - val_accuracy: 0.6733

Epoch 1/10
81/81 [==============================] - 58s 390ms/step - loss: 0.2991 - accuracy: 0.8988 - val_loss: 1.1280 - val_accuracy: 0.6897
Epoch 2/10
81/81 [==============================] - 44s 375ms/step - loss: 0.1001 - accuracy: 0.9705 - val_loss: 1.3414 - val_accuracy: 0.7151
Epoch 3/10
81/81 [==============================] - 44s 373ms/step - loss: 0.0585 - accuracy: 0.9805 - val_loss: 1.8311 - val_accuracy: 0.6987
Epoch 4/10
81/81 [==============================] - 48s 370ms/step - loss: 0.0638 - accuracy: 0.9810 - val_loss: 1.4304 - val_accuracy: 0.6588
Epoch 5/10
81/81 [==============================] - 45s 381ms/step - loss: 0.0791 - accuracy: 0.9712 - val_loss: 1.4285 - val_accuracy: 0.7042
Epoch 6/10
81/81 [==============================] - 46s 373ms/step - loss: 0.0493 - accuracy: 0.9804 - val_loss: 1.4823 - val_accuracy: 0.6987
Epoch 7/10
81/81 [==============================] - 40s 376ms/step - loss: 0.0387 - accuracy: 0.9917 - val_loss: 1.7568 - val_accuracy: 0.6806

In [45]:
weight_cluster_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model_PolynomialDecay50.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model_PolynomialDecay50.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
81/81 [==============================] - 50s 395ms/step - loss: 0.2506 - accuracy: 0.9121 - val_loss: 1.2869 - val_accuracy: 0.6733
Epoch 2/10
81/81 [==============================] - 48s 376ms/step - loss: 0.0468 - accuracy: 0.9842 - val_loss: 1.7616 - val_accuracy: 0.6806
Epoch 3/10
81/81 [==============================] - 44s 376ms/step - loss: 0.0941 - accuracy: 0.9635 - val_loss: 1.6077 - val_accuracy: 0.6515
Epoch 4/10
81/81 [==============================] - 36s 374ms/step - loss: 0.0119 - accuracy: 0.9982 - val_loss: 1.8552 - val_accuracy: 0.7024
Epoch 5/10
81/81 [==============================] - 44s 370ms/step - loss: 0.0702 - accuracy: 0.9741 - val_loss: 1.4057 - val_accuracy: 0.6624
Epoch 6/10
81/81 [==============================] - 46s 369ms/step - loss: 0.0259 - accuracy: 0.9945 - val_loss: 1.7360 - val_accuracy: 0.7005
Epoch 7/10
81/81 [==============================] - 41s 373ms/step - loss: 0.0045 - accuracy: 0.9988 - val_loss: 2.1752 - val_accuracy: 0.6697

Epoch 1/10
81/81 [==============================] - 53s 395ms/step - loss: 0.1740 - accuracy: 0.9352 - val_loss: 1.4683 - val_accuracy: 0.6842
Epoch 2/10
81/81 [==============================] - 48s 376ms/step - loss: 0.0616 - accuracy: 0.9784 - val_loss: 1.6356 - val_accuracy: 0.6860
Epoch 3/10
81/81 [==============================] - 46s 384ms/step - loss: 0.0199 - accuracy: 0.9931 - val_loss: 1.6568 - val_accuracy: 0.6842
Epoch 4/10
81/81 [==============================] - 41s 384ms/step - loss: 0.0692 - accuracy: 0.9791 - val_loss: 1.3034 - val_accuracy: 0.6279
Epoch 5/10
81/81 [==============================] - 40s 376ms/step - loss: 0.0641 - accuracy: 0.9779 - val_loss: 1.5635 - val_accuracy: 0.7060
Epoch 6/10
81/81 [==============================] - 42s 377ms/step - loss: 0.0130 - accuracy: 0.9966 - val_loss: 1.9374 - val_accuracy: 0.6878
Epoch 7/10
81/81 [==============================] - 45s 380ms/step - loss: 0.0412 - accuracy: 0.9850 - val_loss: 1.2039 - val_accuracy: 0.6461

In [46]:
weight_cluster_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model_ConstantSparsity75.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model_ConstantSparsity75.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
81/81 [==============================] - 62s 400ms/step - loss: 0.6453 - accuracy: 0.7874 - val_loss: 1.1045 - val_accuracy: 0.6298
Epoch 2/10
81/81 [==============================] - 47s 374ms/step - loss: 0.1325 - accuracy: 0.9500 - val_loss: 1.5302 - val_accuracy: 0.6897
Epoch 3/10
81/81 [==============================] - 48s 381ms/step - loss: 0.1392 - accuracy: 0.9515 - val_loss: 1.7672 - val_accuracy: 0.6207
Epoch 4/10
81/81 [==============================] - 48s 381ms/step - loss: 0.1027 - accuracy: 0.9584 - val_loss: 1.5496 - val_accuracy: 0.6497
Epoch 5/10
81/81 [==============================] - 48s 379ms/step - loss: 0.0523 - accuracy: 0.9842 - val_loss: 1.5960 - val_accuracy: 0.6679
Epoch 6/10
81/81 [==============================] - 46s 375ms/step - loss: 0.0778 - accuracy: 0.9774 - val_loss: 1.9448 - val_accuracy: 0.6189
Epoch 7/10
81/81 [==============================] - 45s 375ms/step - loss: 0.1128 - accuracy: 0.9603 - val_loss: 1.4496 - val_accuracy: 0.6425

Epoch 1/10
81/81 [==============================] - 57s 394ms/step - loss: 0.7347 - accuracy: 0.7745 - val_loss: 1.4334 - val_accuracy: 0.6624
Epoch 2/10
81/81 [==============================] - 44s 379ms/step - loss: 0.1206 - accuracy: 0.9621 - val_loss: 1.7751 - val_accuracy: 0.5463
Epoch 3/10
81/81 [==============================] - 36s 374ms/step - loss: 0.1167 - accuracy: 0.9598 - val_loss: 1.4414 - val_accuracy: 0.6189
Epoch 4/10
81/81 [==============================] - 46s 379ms/step - loss: 0.1543 - accuracy: 0.9457 - val_loss: 1.6551 - val_accuracy: 0.6552
Epoch 5/10
81/81 [==============================] - 48s 377ms/step - loss: 0.1162 - accuracy: 0.9606 - val_loss: 1.5988 - val_accuracy: 0.6534
Epoch 6/10
81/81 [==============================] - 47s 375ms/step - loss: 0.0731 - accuracy: 0.9714 - val_loss: 1.7647 - val_accuracy: 0.6352
Epoch 7/10
81/81 [==============================] - 48s 378ms/step - loss: 0.0731 - accuracy: 0.9784 - val_loss: 1.5470 - val_accuracy: 0.6316

In [47]:
weight_cluster_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model_PolynomialDecay75.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model_PolynomialDecay75.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
81/81 [==============================] - 54s 391ms/step - loss: 0.1542 - accuracy: 0.9493 - val_loss: 1.3634 - val_accuracy: 0.7078
Epoch 2/10
81/81 [==============================] - 47s 376ms/step - loss: 0.0482 - accuracy: 0.9820 - val_loss: 1.2896 - val_accuracy: 0.7042
Epoch 3/10
81/81 [==============================] - 47s 370ms/step - loss: 0.0305 - accuracy: 0.9910 - val_loss: 1.6926 - val_accuracy: 0.7132
Epoch 4/10
81/81 [==============================] - 37s 381ms/step - loss: 0.0209 - accuracy: 0.9910 - val_loss: 1.7707 - val_accuracy: 0.6897
Epoch 5/10
81/81 [==============================] - 48s 373ms/step - loss: 0.0777 - accuracy: 0.9761 - val_loss: 2.0190 - val_accuracy: 0.6951
Epoch 6/10
81/81 [==============================] - 43s 376ms/step - loss: 0.0165 - accuracy: 0.9927 - val_loss: 2.0250 - val_accuracy: 0.7151
Epoch 7/10
81/81 [==============================] - 48s 371ms/step - loss: 0.0411 - accuracy: 0.9893 - val_loss: 1.5995 - val_accuracy: 0.6951

Epoch 1/10
81/81 [==============================] - 56s 403ms/step - loss: 0.2372 - accuracy: 0.9223 - val_loss: 1.3406 - val_accuracy: 0.6751
Epoch 2/10
81/81 [==============================] - 48s 380ms/step - loss: 0.0543 - accuracy: 0.9834 - val_loss: 1.8108 - val_accuracy: 0.7024
Epoch 3/10
81/81 [==============================] - 45s 384ms/step - loss: 0.0321 - accuracy: 0.9877 - val_loss: 1.9078 - val_accuracy: 0.6987
Epoch 4/10
81/81 [==============================] - 47s 380ms/step - loss: 0.0661 - accuracy: 0.9777 - val_loss: 1.5442 - val_accuracy: 0.7187
Epoch 5/10
81/81 [==============================] - 48s 377ms/step - loss: 0.0149 - accuracy: 0.9959 - val_loss: 2.0381 - val_accuracy: 0.6987
Epoch 6/10
81/81 [==============================] - 41s 382ms/step - loss: 0.0227 - accuracy: 0.9931 - val_loss: 1.8784 - val_accuracy: 0.6515
Epoch 7/10
81/81 [==============================] - 39s 386ms/step - loss: 0.0663 - accuracy: 0.9773 - val_loss: 1.7362 - val_accuracy: 0.6878

In [48]:
weight_cluster_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model_ConstantSparsity90.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model_ConstantSparsity90.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
81/81 [==============================] - 63s 401ms/step - loss: 2.3905 - accuracy: 0.2631 - val_loss: 1.5726 - val_accuracy: 0.2432
Epoch 2/10
81/81 [==============================] - 41s 381ms/step - loss: 1.6067 - accuracy: 0.2478 - val_loss: 1.6718 - val_accuracy: 0.2668
Epoch 3/10
81/81 [==============================] - 44s 386ms/step - loss: 1.5157 - accuracy: 0.3296 - val_loss: 1.4042 - val_accuracy: 0.3702
Epoch 4/10
81/81 [==============================] - 49s 386ms/step - loss: 1.3564 - accuracy: 0.3934 - val_loss: 1.3073 - val_accuracy: 0.4283
Epoch 5/10
81/81 [==============================] - 47s 395ms/step - loss: 1.3717 - accuracy: 0.3851 - val_loss: 1.2981 - val_accuracy: 0.3938
Epoch 6/10
81/81 [==============================] - 39s 384ms/step - loss: 1.3034 - accuracy: 0.4304 - val_loss: 1.2957 - val_accuracy: 0.4610
Epoch 7/10
81/81 [==============================] - 48s 383ms/step - loss: 1.3548 - accuracy: 0.3826 - val_loss: 1.2533 - val_accuracy: 0.4955

Epoch 1/10
81/81 [==============================] - 56s 404ms/step - loss: 3.4403 - accuracy: 0.2560 - val_loss: 1.6343 - val_accuracy: 0.2668
Epoch 2/10
81/81 [==============================] - 49s 392ms/step - loss: 1.6339 - accuracy: 0.2359 - val_loss: 1.5990 - val_accuracy: 0.3158
Epoch 3/10
81/81 [==============================] - 46s 392ms/step - loss: 1.5908 - accuracy: 0.2629 - val_loss: 1.7844 - val_accuracy: 0.2123
Epoch 4/10
81/81 [==============================] - 48s 393ms/step - loss: 1.5555 - accuracy: 0.2659 - val_loss: 1.5635 - val_accuracy: 0.2287
Epoch 5/10
81/81 [==============================] - 48s 394ms/step - loss: 1.4941 - accuracy: 0.2987 - val_loss: 1.3922 - val_accuracy: 0.3430
Epoch 6/10
81/81 [==============================] - 50s 396ms/step - loss: 1.4332 - accuracy: 0.3403 - val_loss: 1.3840 - val_accuracy: 0.3575
Epoch 7/10
81/81 [==============================] - 37s 388ms/step - loss: 1.4240 - accuracy: 0.3637 - val_loss: 1.3903 - val_accuracy: 0.4211

In [49]:
weight_cluster_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model_PolynomialDecay90.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='tf_flowers_models/EfficentNetB0_tf_flowers_model_PolynomialDecay90.h5', 
                     ds_train=tf_flowers_datasets[0], 
                     ds_validation=tf_flowers_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
81/81 [==============================] - 58s 404ms/step - loss: 0.1535 - accuracy: 0.9489 - val_loss: 1.5108 - val_accuracy: 0.6751
Epoch 2/10
81/81 [==============================] - 45s 383ms/step - loss: 0.0613 - accuracy: 0.9787 - val_loss: 1.5013 - val_accuracy: 0.6878
Epoch 3/10
81/81 [==============================] - 44s 382ms/step - loss: 0.0422 - accuracy: 0.9873 - val_loss: 1.4356 - val_accuracy: 0.6751
Epoch 4/10
81/81 [==============================] - 37s 386ms/step - loss: 0.0328 - accuracy: 0.9897 - val_loss: 1.7570 - val_accuracy: 0.6770
Epoch 5/10
81/81 [==============================] - 39s 383ms/step - loss: 0.0396 - accuracy: 0.9873 - val_loss: 2.0132 - val_accuracy: 0.6842
Epoch 6/10
81/81 [==============================] - 44s 380ms/step - loss: 0.0557 - accuracy: 0.9839 - val_loss: 1.6410 - val_accuracy: 0.6788
Epoch 7/10
81/81 [==============================] - 41s 381ms/step - loss: 0.0119 - accuracy: 0.9976 - val_loss: 1.9109 - val_accuracy: 0.6733

Epoch 1/10
81/81 [==============================] - 57s 394ms/step - loss: 0.1448 - accuracy: 0.9548 - val_loss: 1.3962 - val_accuracy: 0.6534
Epoch 2/10
81/81 [==============================] - 46s 381ms/step - loss: 0.0921 - accuracy: 0.9671 - val_loss: 1.6156 - val_accuracy: 0.6770
Epoch 3/10
81/81 [==============================] - 36s 380ms/step - loss: 0.0402 - accuracy: 0.9845 - val_loss: 1.5294 - val_accuracy: 0.6788
Epoch 4/10
81/81 [==============================] - 46s 379ms/step - loss: 0.0523 - accuracy: 0.9858 - val_loss: 1.7826 - val_accuracy: 0.6878
Epoch 5/10
81/81 [==============================] - 48s 376ms/step - loss: 0.0066 - accuracy: 0.9984 - val_loss: 1.7093 - val_accuracy: 0.6788
Epoch 6/10
81/81 [==============================] - 44s 379ms/step - loss: 0.0223 - accuracy: 0.9938 - val_loss: 1.8165 - val_accuracy: 0.7114
Epoch 7/10
81/81 [==============================] - 46s 382ms/step - loss: 0.0467 - accuracy: 0.9875 - val_loss: 1.7841 - val_accuracy: 0.7042

#### beans models

In [50]:
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
33/33 [==============================] - 41s 1s/step - loss: 1.1039 - accuracy: 0.3207 - val_loss: 1.0992 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 35s 982ms/step - loss: 1.0991 - accuracy: 0.3290 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 35s 988ms/step - loss: 1.0986 - accuracy: 0.3376 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 35s 997ms/step - loss: 1.0987 - accuracy: 0.3340 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 35s 986ms/step - loss: 1.0986 - accuracy: 0.3371 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 6/10
33/33 [==============================] - 35s 988ms/step - loss: 1.0985 - accuracy: 0.3591 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 7/10
33/33 [==============================] - 35s 997ms/step - loss: 1.0986 - accuracy: 0.3403 - val_loss: 1.0986 - val_accuracy: 0.3383
Ep

Epoch 1/10
33/33 [==============================] - 45s 1s/step - loss: 1.1105 - accuracy: 0.3065 - val_loss: 1.0991 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 35s 987ms/step - loss: 1.0982 - accuracy: 0.3481 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 35s 996ms/step - loss: 1.0987 - accuracy: 0.3258 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 35s 985ms/step - loss: 1.0987 - accuracy: 0.3469 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 35s 995ms/step - loss: 1.0986 - accuracy: 0.3394 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 6/10
33/33 [==============================] - 35s 979ms/step - loss: 1.0987 - accuracy: 0.3240 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 7/10
33/33 [==============================] - 35s 996ms/step - loss: 1.0987 - accuracy: 0.3450 - val_loss: 1.0986 - val_accuracy: 0.3383
Ep

In [51]:
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
33/33 [==============================] - 41s 1s/step - loss: 1.1317 - accuracy: 0.3409 - val_loss: 1.1018 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 35s 975ms/step - loss: 1.1028 - accuracy: 0.3138 - val_loss: 1.0991 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 36s 995ms/step - loss: 1.0982 - accuracy: 0.3315 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 35s 984ms/step - loss: 1.0988 - accuracy: 0.3213 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 35s 993ms/step - loss: 1.0982 - accuracy: 0.3662 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 6/10
33/33 [==============================] - 35s 989ms/step - loss: 1.0988 - accuracy: 0.3240 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 7/10
33/33 [==============================] - 35s 978ms/step - loss: 1.0989 - accuracy: 0.3102 - val_loss: 1.0986 - val_accuracy: 0.3383
Ep

Epoch 1/10
33/33 [==============================] - 40s 1s/step - loss: 1.1587 - accuracy: 0.3231 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 35s 978ms/step - loss: 1.0996 - accuracy: 0.3489 - val_loss: 1.1012 - val_accuracy: 0.3383
Epoch 3/10
33/33 [==============================] - 35s 982ms/step - loss: 1.1021 - accuracy: 0.3342 - val_loss: 1.0993 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 35s 969ms/step - loss: 1.0990 - accuracy: 0.3355 - val_loss: 1.0994 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 35s 975ms/step - loss: 1.1029 - accuracy: 0.3388 - val_loss: 1.0987 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 35s 976ms/step - loss: 1.1014 - accuracy: 0.2967 - val_loss: 1.0994 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 34s 971ms/step - loss: 1.1017 - accuracy: 0.2912 - val_loss: 1.0991 - val_accuracy: 0.3308
Ep

In [52]:
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
33/33 [==============================] - 42s 1s/step - loss: 1.1224 - accuracy: 0.3349 - val_loss: 1.0987 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 34s 968ms/step - loss: 1.0986 - accuracy: 0.3431 - val_loss: 1.0987 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 35s 977ms/step - loss: 1.0986 - accuracy: 0.3348 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 35s 975ms/step - loss: 1.0987 - accuracy: 0.3064 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 35s 980ms/step - loss: 1.0985 - accuracy: 0.3600 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 35s 981ms/step - loss: 1.0987 - accuracy: 0.3045 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 7/10
33/33 [==============================] - 35s 988ms/step - loss: 1.0986 - accuracy: 0.3176 - val_loss: 1.0986 - val_accuracy: 0.3308
Ep

Epoch 1/10
33/33 [==============================] - 40s 1s/step - loss: 1.1242 - accuracy: 0.3244 - val_loss: 1.0989 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 35s 990ms/step - loss: 1.0987 - accuracy: 0.3386 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 35s 992ms/step - loss: 1.0987 - accuracy: 0.3115 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 35s 984ms/step - loss: 1.0986 - accuracy: 0.3554 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 35s 990ms/step - loss: 1.0987 - accuracy: 0.3147 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 35s 984ms/step - loss: 1.0987 - accuracy: 0.3213 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 35s 997ms/step - loss: 1.0986 - accuracy: 0.3302 - val_loss: 1.0986 - val_accuracy: 0.3383
Ep

In [53]:
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
33/33 [==============================] - 42s 1s/step - loss: 1.1710 - accuracy: 0.3489 - val_loss: 1.1000 - val_accuracy: 0.3383
Epoch 2/10
33/33 [==============================] - 35s 992ms/step - loss: 1.1016 - accuracy: 0.3462 - val_loss: 1.0997 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 35s 981ms/step - loss: 1.0995 - accuracy: 0.3368 - val_loss: 1.0991 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 35s 975ms/step - loss: 1.0994 - accuracy: 0.3389 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 35s 983ms/step - loss: 1.0998 - accuracy: 0.3424 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 34s 966ms/step - loss: 1.1009 - accuracy: 0.3063 - val_loss: 1.0987 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 35s 989ms/step - loss: 1.0991 - accuracy: 0.3575 - val_loss: 1.0986 - val_accuracy: 0.3383
Ep

Epoch 1/10
33/33 [==============================] - 40s 995ms/step - loss: 1.1848 - accuracy: 0.3321 - val_loss: 1.1057 - val_accuracy: 0.3383
Epoch 2/10
33/33 [==============================] - 35s 981ms/step - loss: 1.1063 - accuracy: 0.3253 - val_loss: 1.0993 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 35s 979ms/step - loss: 1.1010 - accuracy: 0.3336 - val_loss: 1.0999 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 35s 981ms/step - loss: 1.0999 - accuracy: 0.3432 - val_loss: 1.1010 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 35s 984ms/step - loss: 1.1002 - accuracy: 0.3402 - val_loss: 1.0992 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 34s 976ms/step - loss: 1.1013 - accuracy: 0.3114 - val_loss: 1.0993 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 35s 982ms/step - loss: 1.1025 - accuracy: 0.3116 - val_loss: 1.0994 - val_accuracy: 0.3383

In [54]:
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
33/33 [==============================] - 41s 1s/step - loss: 1.1004 - accuracy: 0.3367 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 35s 989ms/step - loss: 1.0987 - accuracy: 0.3339 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 35s 989ms/step - loss: 1.0988 - accuracy: 0.3203 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 35s 989ms/step - loss: 1.0986 - accuracy: 0.3404 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 35s 984ms/step - loss: 1.0985 - accuracy: 0.3358 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 6/10
33/33 [==============================] - 36s 995ms/step - loss: 1.0986 - accuracy: 0.3262 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 7/10
33/33 [==============================] - 35s 982ms/step - loss: 1.0984 - accuracy: 0.3688 - val_loss: 1.0986 - val_accuracy: 0.3383
Ep

Epoch 1/10
33/33 [==============================] - 40s 1s/step - loss: 1.1009 - accuracy: 0.3338 - val_loss: 1.0987 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 35s 976ms/step - loss: 1.0990 - accuracy: 0.3247 - val_loss: 1.0987 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 35s 988ms/step - loss: 1.0987 - accuracy: 0.3308 - val_loss: 1.0986 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 35s 981ms/step - loss: 1.0987 - accuracy: 0.3139 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 35s 989ms/step - loss: 1.0987 - accuracy: 0.3383 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 6/10
33/33 [==============================] - 36s 990ms/step - loss: 1.0986 - accuracy: 0.3414 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 7/10
33/33 [==============================] - 35s 979ms/step - loss: 1.0986 - accuracy: 0.3531 - val_loss: 1.0986 - val_accuracy: 0.3383
Ep

In [55]:
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/MobileNetV2_beans_model_PolynomialDecay90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
33/33 [==============================] - 44s 1s/step - loss: 1.1228 - accuracy: 0.3317 - val_loss: 1.0987 - val_accuracy: 0.3383
Epoch 2/10
33/33 [==============================] - 35s 983ms/step - loss: 1.1007 - accuracy: 0.3360 - val_loss: 1.0987 - val_accuracy: 0.3383
Epoch 3/10
33/33 [==============================] - 35s 982ms/step - loss: 1.1018 - accuracy: 0.2843 - val_loss: 1.0988 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 35s 993ms/step - loss: 1.1002 - accuracy: 0.2802 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 35s 988ms/step - loss: 1.1002 - accuracy: 0.3038 - val_loss: 1.0989 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 35s 986ms/step - loss: 1.0999 - accuracy: 0.3081 - val_loss: 1.0989 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 36s 1s/step - loss: 1.0987 - accuracy: 0.3469 - val_loss: 1.0987 - val_accuracy: 0.3308
Epoch

Epoch 1/10
33/33 [==============================] - 40s 1s/step - loss: 1.1353 - accuracy: 0.3375 - val_loss: 1.0995 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 35s 979ms/step - loss: 1.0989 - accuracy: 0.3408 - val_loss: 1.0990 - val_accuracy: 0.3383
Epoch 3/10
33/33 [==============================] - 35s 979ms/step - loss: 1.1015 - accuracy: 0.3095 - val_loss: 1.1013 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 35s 980ms/step - loss: 1.1011 - accuracy: 0.3288 - val_loss: 1.0988 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 35s 978ms/step - loss: 1.1025 - accuracy: 0.2987 - val_loss: 1.1003 - val_accuracy: 0.3383
Epoch 6/10
33/33 [==============================] - 35s 975ms/step - loss: 1.1010 - accuracy: 0.3169 - val_loss: 1.0991 - val_accuracy: 0.3383
Epoch 7/10
33/33 [==============================] - 35s 994ms/step - loss: 1.0996 - accuracy: 0.3392 - val_loss: 1.0988 - val_accuracy: 0.3308
Ep

In [56]:
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
33/33 [==============================] - 59s 1s/step - loss: 1.8269 - accuracy: 0.3106 - val_loss: 1.1011 - val_accuracy: 0.3383
Epoch 2/10
33/33 [==============================] - 48s 1s/step - loss: 1.1046 - accuracy: 0.3387 - val_loss: 1.0977 - val_accuracy: 0.3383
Epoch 3/10
33/33 [==============================] - 48s 1s/step - loss: 1.0988 - accuracy: 0.3462 - val_loss: 1.0989 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 48s 1s/step - loss: 1.0999 - accuracy: 0.3432 - val_loss: 1.0980 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 1.1006 - accuracy: 0.3380 - val_loss: 1.0991 - val_accuracy: 0.3835
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 1.1013 - accuracy: 0.3213 - val_loss: 1.1080 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 1.1017 - accuracy: 0.3330 - val_loss: 1.0974 - val_accuracy: 0.3308
Epoch 8/10
33/33 [==

Epoch 1/10
33/33 [==============================] - 61s 1s/step - loss: 1.9167 - accuracy: 0.3394 - val_loss: 1.1168 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 47s 1s/step - loss: 1.0985 - accuracy: 0.3536 - val_loss: 1.1020 - val_accuracy: 0.3383
Epoch 3/10
33/33 [==============================] - 47s 1s/step - loss: 1.1024 - accuracy: 0.3281 - val_loss: 1.0985 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 1.1024 - accuracy: 0.3416 - val_loss: 1.0984 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 1.1000 - accuracy: 0.3378 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 6/10
33/33 [==============================] - 46s 1s/step - loss: 1.1010 - accuracy: 0.3309 - val_loss: 1.1010 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 1.1009 - accuracy: 0.3236 - val_loss: 1.1022 - val_accuracy: 0.3383
Epoch 8/10
33/33 [==

In [57]:
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay50.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
33/33 [==============================] - 56s 1s/step - loss: 1.7997 - accuracy: 0.3079 - val_loss: 1.1019 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 47s 1s/step - loss: 1.1048 - accuracy: 0.3074 - val_loss: 1.0991 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 46s 1s/step - loss: 1.0981 - accuracy: 0.3554 - val_loss: 1.1009 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 1.1093 - accuracy: 0.3166 - val_loss: 1.1011 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 1.1035 - accuracy: 0.3393 - val_loss: 1.1000 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 46s 1s/step - loss: 1.1029 - accuracy: 0.3064 - val_loss: 1.1013 - val_accuracy: 0.3383
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 1.1036 - accuracy: 0.3358 - val_loss: 1.0985 - val_accuracy: 0.3308
Epoch 8/10
33/33 [==

Epoch 1/10
33/33 [==============================] - 57s 1s/step - loss: 1.7988 - accuracy: 0.3082 - val_loss: 1.1040 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 47s 1s/step - loss: 1.0999 - accuracy: 0.3423 - val_loss: 1.0979 - val_accuracy: 0.3534
Epoch 3/10
33/33 [==============================] - 47s 1s/step - loss: 1.1106 - accuracy: 0.3386 - val_loss: 1.0991 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 1.1015 - accuracy: 0.3459 - val_loss: 1.1007 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 1.1024 - accuracy: 0.3265 - val_loss: 1.1012 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 1.1037 - accuracy: 0.3450 - val_loss: 1.1005 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 1.1049 - accuracy: 0.3197 - val_loss: 1.0988 - val_accuracy: 0.3383
Epoch 8/10
33/33 [==

In [58]:
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
33/33 [==============================] - 62s 1s/step - loss: 1.2641 - accuracy: 0.3325 - val_loss: 1.0977 - val_accuracy: 0.3383
Epoch 2/10
33/33 [==============================] - 48s 1s/step - loss: 1.1077 - accuracy: 0.3267 - val_loss: 1.1185 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 48s 1s/step - loss: 1.1135 - accuracy: 0.3194 - val_loss: 1.0985 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 1.1050 - accuracy: 0.3496 - val_loss: 1.0981 - val_accuracy: 0.4662
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 1.1040 - accuracy: 0.3390 - val_loss: 1.1000 - val_accuracy: 0.3383
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 1.1036 - accuracy: 0.3474 - val_loss: 1.1019 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 1.1018 - accuracy: 0.3286 - val_loss: 1.0984 - val_accuracy: 0.3308
Epoch 8/10
33/33 [==

Epoch 1/10
33/33 [==============================] - 56s 1s/step - loss: 1.2748 - accuracy: 0.3437 - val_loss: 1.0982 - val_accuracy: 0.3233
Epoch 2/10
33/33 [==============================] - 47s 1s/step - loss: 1.1021 - accuracy: 0.3241 - val_loss: 1.1031 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 46s 1s/step - loss: 1.1050 - accuracy: 0.3148 - val_loss: 1.1025 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 1.1014 - accuracy: 0.3373 - val_loss: 1.1002 - val_accuracy: 0.4887
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 1.1045 - accuracy: 0.3275 - val_loss: 1.0982 - val_accuracy: 0.3985
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 1.1018 - accuracy: 0.3302 - val_loss: 1.0981 - val_accuracy: 0.3459
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 1.1015 - accuracy: 0.3321 - val_loss: 1.1110 - val_accuracy: 0.3383
Epoch 8/10
33/33 [==

In [12]:
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay75.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
33/33 [==============================] - 60s 1s/step - loss: 2.5904 - accuracy: 0.2982 - val_loss: 1.1082 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 48s 1s/step - loss: 1.1009 - accuracy: 0.3406 - val_loss: 1.1000 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 47s 1s/step - loss: 1.1006 - accuracy: 0.3014 - val_loss: 1.0988 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 1.0996 - accuracy: 0.3547 - val_loss: 1.1014 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 1.1038 - accuracy: 0.3205 - val_loss: 1.0995 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 1.1029 - accuracy: 0.3095 - val_loss: 1.1020 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 1.0998 - accuracy: 0.3217 - val_loss: 1.1011 - val_accuracy: 0.3383
Epoch 8/10
33/33 [==

Epoch 1/10
33/33 [==============================] - 57s 1s/step - loss: 2.7391 - accuracy: 0.3164 - val_loss: 1.1092 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 47s 1s/step - loss: 1.1040 - accuracy: 0.3272 - val_loss: 1.0997 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 47s 1s/step - loss: 1.1011 - accuracy: 0.3318 - val_loss: 1.0983 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 1.1015 - accuracy: 0.3331 - val_loss: 1.0983 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 1.1001 - accuracy: 0.3057 - val_loss: 1.0988 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 1.0990 - accuracy: 0.3780 - val_loss: 1.1061 - val_accuracy: 0.3383
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 1.1071 - accuracy: 0.3168 - val_loss: 1.1012 - val_accuracy: 0.3383
Epoch 8/10
33/33 [==

In [13]:
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
33/33 [==============================] - 57s 1s/step - loss: 1.3615 - accuracy: 0.3428 - val_loss: 1.0997 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 47s 1s/step - loss: 1.1013 - accuracy: 0.3244 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 3/10
33/33 [==============================] - 48s 1s/step - loss: 1.0989 - accuracy: 0.3402 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 1.0987 - accuracy: 0.3317 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 1.0994 - accuracy: 0.3246 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 1.0986 - accuracy: 0.3356 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 1.0985 - accuracy: 0.3495 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 8/10
33/33 [==

Epoch 1/10
33/33 [==============================] - 57s 1s/step - loss: 1.6190 - accuracy: 0.3401 - val_loss: 1.0986 - val_accuracy: 0.3383
Epoch 2/10
33/33 [==============================] - 47s 1s/step - loss: 1.1025 - accuracy: 0.3318 - val_loss: 1.1047 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 47s 1s/step - loss: 1.1054 - accuracy: 0.3249 - val_loss: 1.1036 - val_accuracy: 0.3383
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 1.1040 - accuracy: 0.3652 - val_loss: 1.1098 - val_accuracy: 0.3308
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 1.1097 - accuracy: 0.2958 - val_loss: 1.1026 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 1.1034 - accuracy: 0.3229 - val_loss: 1.1020 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 1.1053 - accuracy: 0.3164 - val_loss: 1.0994 - val_accuracy: 0.3308
Epoch 8/10
33/33 [==

In [14]:
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=32)
weight_cluster_model(model_path='beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay90.h5', 
                     ds_train=beans_datasets[0], 
                     ds_validation=beans_datasets[1], 
                     batch_size=BATCH_SIZE, 
                     epochs=PRUNING_EPOCHS, 
                     number_of_clusters=128)

Epoch 1/10
33/33 [==============================] - 57s 1s/step - loss: 1.2484 - accuracy: 0.3550 - val_loss: 1.0953 - val_accuracy: 0.3308
Epoch 2/10
33/33 [==============================] - 47s 1s/step - loss: 1.0996 - accuracy: 0.3213 - val_loss: 1.1072 - val_accuracy: 0.3308
Epoch 3/10
33/33 [==============================] - 47s 1s/step - loss: 1.0974 - accuracy: 0.3552 - val_loss: 1.0914 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 1.0909 - accuracy: 0.3628 - val_loss: 1.0793 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 1.0805 - accuracy: 0.3886 - val_loss: 1.0847 - val_accuracy: 0.3759
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 1.0923 - accuracy: 0.3869 - val_loss: 1.0863 - val_accuracy: 0.3609
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 1.0765 - accuracy: 0.4173 - val_loss: 1.0817 - val_accuracy: 0.3835
Epoch 8/10
33/33 [==

Epoch 1/10
33/33 [==============================] - 57s 1s/step - loss: 1.3820 - accuracy: 0.3295 - val_loss: 1.0999 - val_accuracy: 0.3383
Epoch 2/10
33/33 [==============================] - 47s 1s/step - loss: 1.1035 - accuracy: 0.3168 - val_loss: 1.0958 - val_accuracy: 0.3534
Epoch 3/10
33/33 [==============================] - 47s 1s/step - loss: 1.1006 - accuracy: 0.3344 - val_loss: 1.0970 - val_accuracy: 0.3308
Epoch 4/10
33/33 [==============================] - 47s 1s/step - loss: 1.1015 - accuracy: 0.3018 - val_loss: 1.0952 - val_accuracy: 0.3383
Epoch 5/10
33/33 [==============================] - 47s 1s/step - loss: 1.1050 - accuracy: 0.3128 - val_loss: 1.0939 - val_accuracy: 0.3308
Epoch 6/10
33/33 [==============================] - 47s 1s/step - loss: 1.0953 - accuracy: 0.3480 - val_loss: 1.0923 - val_accuracy: 0.3308
Epoch 7/10
33/33 [==============================] - 47s 1s/step - loss: 1.0954 - accuracy: 0.3664 - val_loss: 1.0635 - val_accuracy: 0.3835
Epoch 8/10
33/33 [==

## Quantization of all models

In [15]:
model_paths = []
import os
for file in os.listdir("beans_models/"):
    if file.endswith(".h5"):
        model_paths.append(str(os.path.join("beans_models/", file)))
for file in os.listdir("beans_models_optimized/"):
    if file.endswith(".h5"):
        model_paths.append(str(os.path.join("beans_models_optimized/", file)))
for file in os.listdir("tf_flowers_models/"):
    if file.endswith(".h5"):
        model_paths.append(str(os.path.join("tf_flowers_models/", file)))
for file in os.listdir("tf_flowers_models_optimized/"):
    if file.endswith(".h5"):
        model_paths.append(str(os.path.join("tf_flowers_models_optimized/", file)))
for file in os.listdir("flowers_models/"):
    if file.endswith(".h5"):
        model_paths.append(str(os.path.join("flowers_models/", file)))
for file in os.listdir("flowers_models_optimized/"):
    if file.endswith(".h5"):
        model_paths.append(str(os.path.join("flowers_models_optimized/", file)))

In [16]:
model_paths

['beans_models/EfficentNetB0_beans_model.h5',
 'beans_models/MobileNetV2_beans_model.h5',
 'beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay75.h5',
 'beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity50_KMeansPlusPlus32.h5',
 'beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity90_KMeansPlusPlus32.h5',
 'beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity50_KMeansPlusPlus128.h5',
 'beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay90_KMeansPlusPlus128.h5',
 'beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity75.h5',
 'beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity90_KMeansPlusPlus128.h5',
 'beans_models_optimized/MobileNetV2_beans_model_ConstantSparsity75.h5',
 'beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay90.h5',
 'beans_models_optimized/EfficentNetB0_beans_model_ConstantSparsity75_KMeansPlusPlus32.h5',
 'beans_models_optimized/EfficentNetB0_beans_model_PolynomialDecay75

In [17]:
for model_path in model_paths:
    if "tf_flowers" in model_path:
        model_quantization(model_path=model_path, ds=tf_flowers_datasets[0])
    elif "flowers" in model_path:
        model_quantization(model_path=model_path, ds=flowers_datasets[0])
    elif "beans" in model_path:
        model_quantization(model_path=model_path, ds=beans_datasets[0])

INFO:tensorflow:Assets written to: /tmp/tmpvfj0vvcw/assets


INFO:tensorflow:Assets written to: /tmp/tmpvfj0vvcw/assets


Converted TFLite model (16044956 Bytes) saved to: beans_models_optimized/EfficentNetB0_beans_model.tflite
INFO:tensorflow:Assets written to: /tmp/tmpv7tkgqao/assets


INFO:tensorflow:Assets written to: /tmp/tmpv7tkgqao/assets


Dynamic range quantizatized TFLite model (4789504 Bytes) saved to: beans_models_optimized/EfficentNetB0_beans_model_dynamic_rage_quantization.tflite
INFO:tensorflow:Assets written to: /tmp/tmp8lomock6/assets


...

Full integer quantizatized TFLite model (2975120 Bytes) saved to: flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity90_KMeansPlusPlus128_full_integer_quantization.tflite
INFO:tensorflow:Assets written to: /tmp/tmpipfrp_q_/assets


INFO:tensorflow:Assets written to: /tmp/tmpipfrp_q_/assets


Full integer quantizatized with integer io TFLite model (2973880 Bytes) saved to: flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity90_KMeansPlusPlus128_full_integer_quantization_integer_io.tflite
INFO:tensorflow:Assets written to: /tmp/tmp1h9xp9pm/assets


INFO:tensorflow:Assets written to: /tmp/tmp1h9xp9pm/assets


float16 quantizatized TFLite model (4722624 Bytes) saved to: flowers_models_optimized/MobileNetV2_flowers_model_ConstantSparsity90_KMeansPlusPlus128_float16_quantization.tflite
